# 책정보랑 리뷰데이터 크롤링

# 십진분류/책소개/저자소개/목차/출판사 리뷰/추천사/회원리뷰 column 추가

0528 yes24에서 책소개/저자소개/목차/출판사리뷰/추천사/회원리뷰 크롤링 완료 + 십진분류   
0530 태그 제거 완료 + 듀이십진으로 수정
- 인터파크에서 회원리뷰 없는 도서들 모두 검색 시 -> 인터파크에도 없는 거 확인
- 남은 건 없는 데이터 정보 찾아보기 or 결측치 처리

광운 인기도서 상위 60권 진행(0-59)

In [1]:
import pandas as pd
import numpy as np
import csv
import requests
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv('광운대 인기도서.csv')
df

,서명,저자,발행처,출판연도,이용건수
0,불편한 편의점 : 김호연 장편소설,김호연,나무옆의자,2021-2022,90
1,파친코 : 이민진 장편소설,이민진,문학사상,2018,77
2,(윤성우의 열혈) C 프로그래밍 / 개정판[실은2판],윤성우,오렌지미디어,2010,55
3,"물고기는 존재하지 않는다 : 상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기","Miller, Lulu",곰,2021,54
4,공정하다는 착각 : 능력주의는 모두에게 같은 기회를 제공하는가,"Sandel, Michael J",와이즈베리 : 미래엔,2020,50
...,...,...,...,...,...
183,1Q84 : 무라카미 하루키 장편소설 = Ichi-Kew-Hachi-Yon,촌상춘수(村上春樹),문학동네,2009-2010,11
184,해리포터와 불사조 기사단 = Harry Potter and the Order of ...,"Rowling, J. K.",문학수첩,2003,11
185,(새)먼나라 이웃나라 : 이원복교수님과 함께 떠나는 세계 역사 기행!. 1-20 /...,이원복,김영사,1999-2018,11
186,(생활코딩!) 리액트 프로그래밍,이고잉,위키북스,2021,11


In [3]:
df = df.head(60)
df

,서명,저자,발행처,출판연도,이용건수
0,불편한 편의점 : 김호연 장편소설,김호연,나무옆의자,2021-2022,90
1,파친코 : 이민진 장편소설,이민진,문학사상,2018,77
2,(윤성우의 열혈) C 프로그래밍 / 개정판[실은2판],윤성우,오렌지미디어,2010,55
3,"물고기는 존재하지 않는다 : 상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기","Miller, Lulu",곰,2021,54
4,공정하다는 착각 : 능력주의는 모두에게 같은 기회를 제공하는가,"Sandel, Michael J",와이즈베리 : 미래엔,2020,50
5,생각의 탄생,"Root-Bernstein, Robert",에코의서재,2007,49
6,역행자 : 돈·시간·운명으로부터 완전한 자유를 얻는 7단계 인생 공략집,자청,웅진지식하우스 : 웅진씽크빅,2022,46
7,밑바닥부터 시작하는 딥러닝 = Deep learning from scratch,재등강의(斎藤康毅),한빛미디어,2017-2020,37
8,(윤성우의 열혈) C++ 프로그래밍 / 개정판[실은2판],윤성우,오렌지미디어,2010,35
9,아몬드 : 손원평 장편소설,손원평,창비,2017,35


In [4]:
# 빈 데이터프레임 생성
df2 = pd.DataFrame(columns=['Book Title', 'Author', 'Author info', 'Book Introduce', 
                           'Table of Contents', 'Publisher Review', 'Recommendations', 'Reviews'])
df2['Book Title'] = df['서명']
df2['Author'] = df['저자']
df2

,Book Title,Author,Author info,Book Introduce,Table of Contents,Publisher Review,Recommendations,Reviews
0,불편한 편의점 : 김호연 장편소설,김호연,NaN,NaN,NaN,NaN,NaN,NaN
1,파친코 : 이민진 장편소설,이민진,NaN,NaN,NaN,NaN,NaN,NaN
2,(윤성우의 열혈) C 프로그래밍 / 개정판[실은2판],윤성우,NaN,NaN,NaN,NaN,NaN,NaN
3,"물고기는 존재하지 않는다 : 상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기","Miller, Lulu",NaN,NaN,NaN,NaN,NaN,NaN
4,공정하다는 착각 : 능력주의는 모두에게 같은 기회를 제공하는가,"Sandel, Michael J",NaN,NaN,NaN,NaN,NaN,NaN
5,생각의 탄생,"Root-Bernstein, Robert",NaN,NaN,NaN,NaN,NaN,NaN
6,역행자 : 돈·시간·운명으로부터 완전한 자유를 얻는 7단계 인생 공략집,자청,NaN,NaN,NaN,NaN,NaN,NaN
7,밑바닥부터 시작하는 딥러닝 = Deep learning from scratch,재등강의(斎藤康毅),NaN,NaN,NaN,NaN,NaN,NaN
8,(윤성우의 열혈) C++ 프로그래밍 / 개정판[실은2판],윤성우,NaN,NaN,NaN,NaN,NaN,NaN
9,아몬드 : 손원평 장편소설,손원평,NaN,NaN,NaN,NaN,NaN,NaN


## 연습 - 1. 불편한 편의점 

In [5]:
url = 'http://www.yes24.com/Product/Goods/99308021'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개

In [7]:
# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')



최고의 재미와 감동을 선사하는 K힐링소설의 원조
 
***주요 서점 종합 베스트셀러 1위
***국립중앙도서관 2022 올해의 책
***전국 37개 도시 올해의 책
***해외 18개국 판권 수출
***연극 [불편한 편의점] 절찬 상연 중

불편한데 자꾸 가고 싶은 편의점이 있다! 
힘들게 살아낸 오늘을 위로하는 편의점의 밤
정체불명의 알바로부터 시작된 웃음과 감동의 나비효과
『망원동 브라더스』 김호연의 ‘동네 이야기’ 시즌 2

2013년 세계문학상 우수상 수상작 『망원동 브라더스』로 데뷔한 후 일상적 현실을 위트 있게 그린 경쾌한 작품과 인간의 내밀한 욕망을 기발한 상상력으로 풀어낸 스릴러 장르를 오가며 독자적인 작품 세계를 쌓아올린 작가 김호연. 그의 다섯 번째 장편소설 『불편한 편의점』이 출간되었다. 『불편한 편의점』은 청파동 골목 모퉁이에 자리 잡은 작은 편의점을 무대로 힘겨운 시대를 살아가는 우리 이웃들의 삶의 속내와 희로애락을 따뜻하고 유머러스하게 담아낸 작품이다. 『망원동 브라더스』에서 망원동이라는 공간의 체험적 지리지를 잘 활용해 유쾌한 재미와 공감을 이끌어냈듯 이번에는 서울의 오래된 동네 청파동에 대한 공감각을 생생하게 포착해 또 하나의 흥미진진한 ‘동네 이야기’를 탄생시켰다. 

서울역에서 노숙인 생활을 하던 독고라는 남자가 어느 날 70대 여성의 지갑을 주워준 인연으로 그녀가 운영하는 편의점에서 야간 알바를 하면서 이야기가 시작된다. 덩치가 곰 같은 이 사내는 알코올성 치매로 과거를 기억하지 못하는 데다 말도 어눌하고 행동도 굼떠 과연 손님을 제대로 상대할 수 있을까 의구심을 갖게 하는데 웬걸, 의외로 그는 일을 꽤 잘해낼 뿐 아니라 주변 사람들을 묘하게 사로잡으면서 편의점의 밤을 지키는 든든한 일꾼이 되어간다. 

현실감 넘치는 캐릭터와 그들 간의 상호작용을 점입가경으로 형상화하는 데 일가견이 있는 작가의 작품답게 이 소설에서도 독특한 개성과 사연을 지닌 인물들이 차례로 등장해 서로 티격태격하며 별난 관계를 형성해간다. 고등학교에서 역사

# 저자소개

In [9]:
# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')

영화·만화·소설을 넘나들며 온갖 이야기를 써나가는 전천후 스토리텔러. 1974년 서울생. 고려대학교 인문대학 국어국문학과를 졸업했다. 첫 직장인 영화사에서 공동 작업한 시나리오 「이중간첩」이 영화화되며 시나리오 작가가 되었다. 두 번째 직장인 출판사에서 만화 기획자로 일하며 쓴 「실험인간지대」가 제1회 부천만화스토리 공모전에서 대상을 수상하며 만화 스토리 작가가 되었다. 같은 출판사 소설 편집자로 남의 소설을 만지다가 급기야 전업 작가로 나섰다. 이후 ‘젊은 날 닥치는 대로 글쓰기’를 실천하던 중 장편소설 『망원동 브라더스』로 2013년 제9회 세계문학상 우수상을 수상하며 소설가가 되었다.장편소설 『망원동 브라더스』(2013), 『연적』(2015), 『고스트라이터즈』(2017), 『파우스터』(2019)와 산문집 『매일 쓰고 다시 쓰고 끝까지 씁니다』(2020)를 펴냈고, 영화 「이중간첩」(2003), 「태양을 쏴라」(2015)의 시나리오와 「남한산성」(2017)의 기획에 참여했다. 2021년 『망원동 브라더스』에 이은 ‘동네 이야기’ 시즌 2 『불편한 편의점』을 출간했다.


# 목차

In [12]:
# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')

산해진미 도시락 
제이에스 오브 제이에스 
삼각김밥의 용도 
원 플러스 원 
불편한 편의점 
네 캔에 만 원 
폐기 상품이지만 아직 괜찮아 
ALWAYS 

감사의 글 



# 출판사리뷰

In [13]:
# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')

원 플러스 원의 기쁨, 삼각김밥 모양의 슬픔, 만 원에 네 번의 폭소가 터지는 곳!힘겨운 시대를 살아가는 우리들에게 다가온 조금 특별한 편의점 이야기 2013년 세계문학상 우수상 수상작 『망원동 브라더스』로 데뷔한 후 일상적 현실을 위트 있게 그린 경쾌한 작품과 인간의 내밀한 욕망을 기발한 상상력으로 풀어낸 스릴러 장르를 오가며 독자적인 작품 세계를 쌓아올린 작가 김호연. 그의 다섯 번째 장편소설 『불편한 편의점』이 나무옆의자에서 출간되었다. 『불편한 편의점』은 청파동 골목 모퉁이에 자리 잡은 작은 편의점을 무대로 힘겨운 시대를 살아가는 우리 이웃들의 삶의 속내와 희로애락을 따뜻하고 유머러스하게 담아낸 작품이다. 『망원동 브라더스』에서 망원동이라는 공간의 체험적 지리지를 잘 활용해 유쾌한 재미와 공감을 이끌어냈듯 이번에는 서울의 오래된 동네 청파동에 대한 공감각을 생생하게 포착해 또 하나의 흥미진진한 ‘동네 이야기’를 탄생시켰다. 서울역에서 노숙인 생활을 하던 독고라는 남자가 어느 날 70대 여성의 지갑을 주워준 인연으로 그녀가 운영하는 편의점에서 야간 알바를 하면서 이야기가 시작된다. 덩치가 곰 같은 이 사내는 알코올성 치매로 과거를 기억하지 못하는 데다 말도 어눌하고 행동도 굼떠 과연 손님을 제대로 상대할 수 있을까 의구심을 갖게 하는데 웬걸, 의외로 그는 일을 꽤 잘해낼 뿐 아니라 주변 사람들을 묘하게 사로잡으면서 편의점의 밤을 지키는 든든한 일꾼이 되어간다. 현실감 넘치는 캐릭터와 그들 간의 상호작용을 점입가경으로 형상화하는 데 일가견이 있는 작가의 작품답게 이 소설에서도 독특한 개성과 사연을 지닌 인물들이 차례로 등장해 서로 티격태격하며 별난 관계를 형성해간다. 고등학교에서 역사를 가르치다 정년퇴임하여 매사에 교사 본능이 발동하는 편의점 사장 염 여사를 필두로 20대 취준생 알바 시현, 50대 생계형 알바 오 여사, 매일 밤 야외 테이블에서 참참참(참깨라면, 참치김밥, 참이슬) 세트로 혼술을 하며 하루의 스트레스를 푸는 회사원 경만, 마지막이라는 각오

# 추천평

In [14]:
# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')

서울역 홈리스로 지내면서도 자기의 안위보다는 지갑을 잃어버린 낯선 부인의 안부를 걱정하는 독고 씨. 그런 독고를 향해 우정과 치유의 손길을 내미는 편의점 사장 염 여사. 두 사람의 아름다운 우정의 역사는 코로나 사태 이후 고독과 불안을 더욱 예민하게 느끼게 된 우리들에게 눈부신 영감의 씨앗을 심어준다. 모두가 무시하고, 외면하고, 회피하던 홈리스 독고 씨의 변신은 어쩌면 덜 놀라운 사실이다. 독고 씨의 진짜 재능은 많은 사람을 너끈히 구할 수 있는 눈물겹도록 따스한 마음이기에. 

- 정여울 (『1일 1페이지 세상에서 가장 짧은 심리수업 365』 저자)


# 회원리뷰 5페이지

In [106]:
# 회원리뷰
url_list = [
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685450170992',
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=2&Type=ALL&DojungAfterBuy=0&_=1685450170993',
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=3&Type=ALL&DojungAfterBuy=0&_=1685450170994',
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=4&Type=ALL&DojungAfterBuy=0&_=1685450170995',
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=5&Type=ALL&DojungAfterBuy=0&_=1685450170996'
]

for url in url_list:
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')

    for review in soup.select('div.review_cont'):
        print(review.get_text(strip=True))
        rev = review.get_text(strip=True)
df2['Reviews'] = rev

직원은 불편하지만 손님은 편한 편의점.
제목에서 느낄 수 있었던 건 편의점은 단순히 물건을 파는데 그치는 것만이 아닌 서비스와 손님에 대한 관심 더 나아가 사랑을 파는 곳이 아닌가 하는 생각이 들게 했다.
노숙자이며 희망도 없고 생을 마감하고자 했던 독고씨에게 살아갈 희망과 일자리를 주었던 곳, 매출보다는 직원들에게 일자리를 제공하고 그 가족까지도...더보기
직원은 불편하지만 손님은 편한 편의점.제목에서 느낄 수 있었던 건 편의점은 단순히 물건을 파는데 그치는 것만이 아닌 서비스와 손님에 대한 관심 더 나아가 사랑을 파는 곳이 아닌가 하는 생각이 들게 했다.노숙자이며 희망도 없고 생을 마감하고자 했던 독고씨에게 살아갈 희망과 일자리를 주었던 곳, 매출보다는 직원들에게 일자리를 제공하고 그 가족까지도 살피는 사장님의 경영철학과  타인을 대하는 사랑하는 마음이 전달되어 각자 아픈 상처가 있던 사람들의 마음을 어루만져주고 변화시켜가는 따뜻한 이야기가 담겨 있어 울다가 웃다가를 반복하며 읽은 감동적인 책이었다.남녀노소가 모두 들러 자기의 필요를 채우는 그 작은 곳이 여러 사람들의 애환을 담고 있는 작은 사회라는 생각이 들었다. 잠깐 들러 필요한 물건만을 사가는 상점이 아니라 서로 연결되어 가족 같은 정이 느껴지는 곳이다.지금은 점점 사라져가는 동네슈퍼는 동네 사람들의 일상을 나누는 곳이기도 했다.외상도 있고 사람 냄새나는 정이 있었던 곳. 관심과 애정을 파는 곳주위에  많이 있는 편의점들도 그런 곳이 되기를 바래본다.
5월 오프라인 독서모임에서 읽게 된 불편한 편의점!

불편한편의점 2를먼저 읽고 읽은 책이라 그런지 익숙한 주인공들이 나와서 반가웠다.
서울역에서 노숙자 생활을 하던 독고의 변화도 인상적이었고, 독고로 인해서 편의점에 방문하는 사람들이 변화하는 모습도 보기 좋았다. 우리 주변에서 흔히 있을 법한 이야기들로 이루어져 있어서 낯설지 않았고 책을 읽으면...더보기
5월 오프라인 독서모임에서 읽게 된 불편한 편의점!불편한 편의점 2를 먼저 읽고 읽은 책이라 그

잔잔 감동과 재미가 있는  좋은책입니다.읽으며 행복한 기분이었습니다.단숨에 읽히는 책입니다.교훈도 있고  재미도 있고 편의점 사장님 같은 사람이 되고싶네요주인공의 무뚝뚝하고 속깊은 마음이 멋집니다.동생에게 선물도 했답니다.표지도 너무나 아름답습니다.어느동네엔가 있을것 같은 편의점 풍경인데책을 읽고보니 더 예쁘게 보이네요,   작가의 다른 책도...더보기
잔잔 감동과 재미가 있는  좋은책입니다.읽으며 행복한 기분이었습니다.단숨에 읽히는 책입니다.교훈도 있고  재미도 있고편의점 사장님 같은 사람이 되고싶네요주인공의 무뚝뚝하고 속깊은 마음이 멋집니다.동생에게 선물도 했답니다.표지도 너무나 아름답습니다.어느동네엔가 있을것 같은 편의점 풍경인데책을 읽고보니 더 예쁘게 보이네요,작가의 다른 책도 궁긍하고  기대되어 다른책도 읽고싶어요
학교 독서시간에 읽어보려 한 책, 읽어본 결과 청소년이라면 읽어 볼만 하지만 고등학생부터는 약간 유치할 수도 있어서 초등학교 고학년에서 중학교 저학년이 읽기 적당한 책 같다. 노숙자 아저씨가 점점 변해가는 과정에서 주변사람들에게 끼친 영향 정도가 이 책의 주요 줄거리 같다. 굉장히 최근에 나온 책이라 요즘 시대의 사람들의 생각이 많이 반영된거 같아...더보기
학교 독서시간에 읽어보려 한 책, 읽어본 결과 청소년이라면 읽어 볼만 하지만 고등학생부터는 약간 유치할 수도 있어서 초등학교 고학년에서 중학교 저학년이 읽기 적당한 책 같다. 노숙자 아저씨가 점점 변해가는 과정에서 주변사람들에게 끼친 영향 정도가 이 책의 주요 줄거리 같다. 굉장히 최근에 나온 책이라 요즘 시대의 사람들의 생각이 많이 반영된거 같아 공감도 많이 되었다.
중학생인 아이가 예전부터 읽고 싶다고 했었는데 좀 늦게 구입하게 되었네요. 워낙 베스트셀러로 유명한 책이라 요즘 독서와는 거리가 먼 삶을 살고 있던 저도 읽어보게 되었습니다. 아이가 책을 금새 다 읽고 저에게 넘겨줬는데 역시 재미있고 감동적이어서 저도 바로 다 읽었네요.이 책을 통해서 우리 모두 각자의 삶에 어려움이 있지

# 1 불편한 편의점 [800]

In [112]:
url = 'http://www.yes24.com/Product/Goods/99308021'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
    df2['Book Introduce'] = introduce
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
    df2['Author info'] = intro_text
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
    df2['Table of Contents'] = info_text
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
    df2['Publisher Review'] = rewiew_pub_text
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
    df2['Recommendations'] = res_text
else:
    print('없음')
print("===============================================")

# 회원리뷰
url_list = [
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685450170992',
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=2&Type=ALL&DojungAfterBuy=0&_=1685450170993',
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=3&Type=ALL&DojungAfterBuy=0&_=1685450170994',
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=4&Type=ALL&DojungAfterBuy=0&_=1685450170995',
    'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99308021?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=5&Type=ALL&DojungAfterBuy=0&_=1685450170996'
]

reviews = []  # 리뷰를 저장할 리스트

for url in url_list:
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')

    for review in soup.select('div.review_cont'):
        review_text = review.get_text(strip=True)
        print(review_text)
        reviews.append(review_text)

df2['Reviews'] = reviews



최고의 재미와 감동을 선사하는 K힐링소설의 원조
 
***주요 서점 종합 베스트셀러 1위
***국립중앙도서관 2022 올해의 책
***전국 37개 도시 올해의 책
***해외 18개국 판권 수출
***연극 [불편한 편의점] 절찬 상연 중

불편한데 자꾸 가고 싶은 편의점이 있다! 
힘들게 살아낸 오늘을 위로하는 편의점의 밤
정체불명의 알바로부터 시작된 웃음과 감동의 나비효과
『망원동 브라더스』 김호연의 ‘동네 이야기’ 시즌 2

2013년 세계문학상 우수상 수상작 『망원동 브라더스』로 데뷔한 후 일상적 현실을 위트 있게 그린 경쾌한 작품과 인간의 내밀한 욕망을 기발한 상상력으로 풀어낸 스릴러 장르를 오가며 독자적인 작품 세계를 쌓아올린 작가 김호연. 그의 다섯 번째 장편소설 『불편한 편의점』이 출간되었다. 『불편한 편의점』은 청파동 골목 모퉁이에 자리 잡은 작은 편의점을 무대로 힘겨운 시대를 살아가는 우리 이웃들의 삶의 속내와 희로애락을 따뜻하고 유머러스하게 담아낸 작품이다. 『망원동 브라더스』에서 망원동이라는 공간의 체험적 지리지를 잘 활용해 유쾌한 재미와 공감을 이끌어냈듯 이번에는 서울의 오래된 동네 청파동에 대한 공감각을 생생하게 포착해 또 하나의 흥미진진한 ‘동네 이야기’를 탄생시켰다. 

서울역에서 노숙인 생활을 하던 독고라는 남자가 어느 날 70대 여성의 지갑을 주워준 인연으로 그녀가 운영하는 편의점에서 야간 알바를 하면서 이야기가 시작된다. 덩치가 곰 같은 이 사내는 알코올성 치매로 과거를 기억하지 못하는 데다 말도 어눌하고 행동도 굼떠 과연 손님을 제대로 상대할 수 있을까 의구심을 갖게 하는데 웬걸, 의외로 그는 일을 꽤 잘해낼 뿐 아니라 주변 사람들을 묘하게 사로잡으면서 편의점의 밤을 지키는 든든한 일꾼이 되어간다. 

현실감 넘치는 캐릭터와 그들 간의 상호작용을 점입가경으로 형상화하는 데 일가견이 있는 작가의 작품답게 이 소설에서도 독특한 개성과 사연을 지닌 인물들이 차례로 등장해 서로 티격태격하며 별난 관계를 형성해간다. 고등학교에서 역사

잔잔 감동과 재미가 있는  좋은책입니다.읽으며 행복한 기분이었습니다.단숨에 읽히는 책입니다.교훈도 있고  재미도 있고 편의점 사장님 같은 사람이 되고싶네요주인공의 무뚝뚝하고 속깊은 마음이 멋집니다.동생에게 선물도 했답니다.표지도 너무나 아름답습니다.어느동네엔가 있을것 같은 편의점 풍경인데책을 읽고보니 더 예쁘게 보이네요,   작가의 다른 책도...더보기
잔잔 감동과 재미가 있는  좋은책입니다.읽으며 행복한 기분이었습니다.단숨에 읽히는 책입니다.교훈도 있고  재미도 있고편의점 사장님 같은 사람이 되고싶네요주인공의 무뚝뚝하고 속깊은 마음이 멋집니다.동생에게 선물도 했답니다.표지도 너무나 아름답습니다.어느동네엔가 있을것 같은 편의점 풍경인데책을 읽고보니 더 예쁘게 보이네요,작가의 다른 책도 궁긍하고  기대되어 다른책도 읽고싶어요
학교 독서시간에 읽어보려 한 책, 읽어본 결과 청소년이라면 읽어 볼만 하지만 고등학생부터는 약간 유치할 수도 있어서 초등학교 고학년에서 중학교 저학년이 읽기 적당한 책 같다. 노숙자 아저씨가 점점 변해가는 과정에서 주변사람들에게 끼친 영향 정도가 이 책의 주요 줄거리 같다. 굉장히 최근에 나온 책이라 요즘 시대의 사람들의 생각이 많이 반영된거 같아...더보기
학교 독서시간에 읽어보려 한 책, 읽어본 결과 청소년이라면 읽어 볼만 하지만 고등학생부터는 약간 유치할 수도 있어서 초등학교 고학년에서 중학교 저학년이 읽기 적당한 책 같다. 노숙자 아저씨가 점점 변해가는 과정에서 주변사람들에게 끼친 영향 정도가 이 책의 주요 줄거리 같다. 굉장히 최근에 나온 책이라 요즘 시대의 사람들의 생각이 많이 반영된거 같아 공감도 많이 되었다.
중학생인 아이가 예전부터 읽고 싶다고 했었는데 좀 늦게 구입하게 되었네요. 워낙 베스트셀러로 유명한 책이라 요즘 독서와는 거리가 먼 삶을 살고 있던 저도 읽어보게 되었습니다. 아이가 책을 금새 다 읽고 저에게 넘겨줬는데 역시 재미있고 감동적이어서 저도 바로 다 읽었네요.이 책을 통해서 우리 모두 각자의 삶에 어려움이 있지

ValueError: Length of values (50) does not match length of index (60)

# 2 파친코 : 목차 없음 [800]

In [12]:
url = 'http://www.yes24.com/Product/Goods/110641272'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/110641272?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271066079'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



“역사는 우리를 저버렸지만, 그래도 상관없다”
역사에 외면당한 재일조선인 가족의 대서사극
전 세계를 감동시킨 이민진 작가 화제작 『파친코』 새롭게 출간!

“내게 ‘한국인’은 이야기의 주인공이 될 가치가 있는 이들이다.
나는 가능한 한 오래 한국인 이야기를 쓰고 싶다.” 
- ‘한국 독자들에게’ 중에서

4대에 걸친 재일조선인 가족의 이야기를 그린 세계적 베스트셀러, 이민진 작가의 장편소설 『파친코』가 새롭게 출간되었다. 『파친코』는 재미교포 1.5세대인 이민진 작가가 30년에 달하는 세월에 걸쳐 집필한 대하소설로, 2017년 출간되어 『뉴욕타임스』 베스트셀러에 올랐다. 현재까지 전 세계 33개국에 번역 수출되었으며, BBC, 아마존 등 75개 이상의 주요 매체의 ‘올해의 책’으로 선정되었을 뿐 아니라 전미도서상 최종 후보에 이름을 올리며 평단과 대중을 모두 사로잡은 작품이다. 버락 오바마 전 미국 대통령으로부터 “회복과 연민에 대한 강력한 이야기”라는 찬사를 받으며 주목을 받았다.

지난 4월 판권 계약이 종료되며 절판되었던 『파친코』는 새로운 번역과 디자인으로 한국 독자에게 돌아왔다. 첫 문장(“역사는 우리를 저버렸지만, 그래도 상관없다”)에서부터 원문의 의미를 보다 충실하게 전달하고자 했으며, 작품 특유의 속도감 있는 문체를 살리고자 노력했다. 또한 작가가 처음 의도한 구조와 흐름을 살리기 위해 총 세 파트(1부 ‘고향’, 2부 ‘모국’, 3부 ‘파친코’)로 된 원서의 구성을 그대로 따랐다. 새 출간을 기념해 ‘한국 독자들에게’ 보내는 글에는 한국인 이야기를 계속해서 쓰는 이유를 밝혔다. 작가는 “우리가 매력적이기 때문”이라며, “한국인은 지적으로나, 감성적으로나 깊이 있는 이야기의 주인공이 될 가치가 있는 이들”이기에 앞으로도 한국의 이야기를 젊은 세대들에게 들려주고 싶다며 한국 독자들에게 각별한 애정을 표했다.






                                    

전 세계에서 뜨거운 관심과 사랑을 받고 있는 한국계 미국인 

# 3 (윤성우의 열혈) C 프로그래밍 : 출판사 리뷰, 추천사 없음 [600]

In [13]:
url = 'http://www.yes24.com/Product/Goods/4333686'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/4333686?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271193542'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



2003년도에 출간된 윤성우 저 「열혈강의 C 프로그래밍」의 개정판이다. 출간 이후 가장 많은 독자들이 선택해온 C언어 기본서로서 그 자리를 계속해서 이어가기에 부족함이 없도록 개정되었다.본 도서는 강의가 필요 없을 만큼 쉽게 설명되어 있는 책이다. 하지만 강의가 필요한 독자들을 위해서 저자가 직접 본 도서의 내용을 강의한다. 책만큼 유명한 이 강의와 함께 한다면 누구나 C언어를 쉽게 공부할 수 있을 것이다.그 사이 저자가 여러 서적을 집필하면서 쌓아온 경험과 독자들의 다양한 의견을 바탕으로 새롭게 태어난 이번 개정판은 여러분이 C언어를 공부하는데 있어서 가장 든든한 친구로 함께할 것이다.                     

벤처회사에서 개발자로 일하다가 IT분야의 집필과 강의로 처음 이름이 알려진 그는 2000년대 초반까지는 소프트웨어 아키텍트(Architect)로 일을 하다가, 2004년부터 지금까지 OpenGL-ES 그래픽스 라이브러리의 구현과 3D 가속 칩의 개발 및 크로노스 그룹(모바일 국제 표준화 컨소시엄)의 표준안에 관련된 일에 참여하였다. 또한 핸드폰용 DMB 칩의 개발에도 참여하였으며, 현재는 ㈜액시스소프트의 CTO로 있으면서 웹 기반 솔루션 개발에 관심을 갖고 있다. 시간이 날 때마다 조깅을 하며 다양한 구상을 하는 저자는, 방법론에 근거한 소프트웨어 설계 전문가가 모든 분야에서 활발히 활동하여 소프트웨어 발전에 기여해야 한다는 생각을 갖고 있다.프로그래밍을 공부한다면 모르는 사람이 없을 것 같은 베스트셀러 저자이다. 여전히 쉽게 설명하는 방법에 대해 고민하고 있으며 그 고민 결과를 바탕으로 책을 집필하고 있다. 프로그래머라는 직업에 충실하기위해 적지않은 기간동안 집필활동이 없었지만 그간 축적된 에너지를 바탕으로 다수의 책을 쓸 계획을가지고 집필에 임하고 있다.
Part 01 C언어의 기본
   Chapter 01 이것이 C언어다.
      01-1 C언어의 개론적인 이야기
      01-2 C 프로그램의 완성과정
  
   Cha

# 4 물고기는 존재하지 않는다 [500]

In [14]:
url = 'http://www.yes24.com/Product/Goods/105526047'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/105526047?goodsSortNo=001020&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271288676'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



‘방송계의 퓰리처상’ 피버디상 수상자 룰루 밀러의사랑과 혼돈, 과학적 집착에 관한 경이롭고도 충격적인 데뷔작!‘방송계의 퓰리처상’으로 불리는 피버디상(Peabody Awards)을 수상한 과학 전문기자 룰루 밀러의 경이로운 논픽션 《물고기는 존재하지 않는다》는 여러 언론 매체에서 ‘2020년 최고의 책’으로 선정할 만큼 수많은 찬사를 받은 화제의 베스트셀러다. 집착에 가까울 만큼 자연계에 질서를 부여하려 했던 19세기 어느 과학자의 삶을 흥미롭게 좇아가는 이 책은 어느 순간 독자들을 혼돈의 한복판으로 데려가서 우리가 믿고 있던 삶의 질서에 관해 한 가지 의문을 제기한다. “물고기가 존재하지 않는다는 것은 엄연한 하나의 사실이다. 그렇다면 우리는 또 무엇을 잘못 알고 있을까?” 하고 말이다. 누군가에게는 이 질문이 살아가는 데 아무런 영향을 미치지 않을 수도 있다. 하지만 세상을 바라보는 “진실한 관계들”에 한층 가까이 다가가기 위해 노력하는 사람들에게는 분명 이 책이 놀라운 영감과 어느 한쪽으로도 치우치지 않는 폭넓은 시야를 제공해줄 것이다.물고기가 존재하지 않는다면, 우리가 이 세계에 관해 아직 모르고 있는 것은 또 뭐가 있을까? 또 어떤 범주들이 무너질 참일까? 구름도 생명이 있는 존재일 수 있을까? 누가 알겠는가. 해왕성에서는 다이아몬드가 비로 내린다는데. 그건 정말이다. 바로 몇 년 전에 과학자들이 그 사실을 알아냈다. 우리가 세상을 더 오래 검토할수록 세상은 더 이상한 곳으로 밝혀질 것이다. _265쪽                                           

‘방송계의 퓰리처상’으로 불리는 피버디상(Peabody Awards)을 수상한 과학 전문기자로, 15년 넘게 미국공영라디오방송국(NPR)에서 일하고 있다. 인간의 행동을 형성하는 ‘보이지 않는 힘들’에 관한 이야기를 나누는 NPR의 〈인비저빌리아(Invisibilia)〉의 공동 기획자이고, 뉴욕공영라디오방송국(WNYC)의 〈라디오랩(Radiolab)〉에도 자주 참여하고 있

# 5 공정하다는 착각 [300]

In [15]:
url = 'http://www.yes24.com/Product/Goods/94489333'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/94489333?goodsSortNo=001009&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271355648'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



마이클 샌델 10여 년 만의 신간, 『공정하다는 착각』 출간!
샌델, 기울어진 사회구조 이면에 도사린 ‘능력주의의 덫’을 해체하다

또 다시 ‘공정’이 화두다. 언론 미디어를 통해, 부유층과 빈곤층, 청년과 장년, 정치인의 입을 통해 끊임없이 쏟아져 나온다. 기업은 정규직?비정규직 논란에서 비롯된 ‘공정 채용’ 문제로 혼란에 빠져 있고, 정치권에선 ‘공정경제3법’과 ‘재난지원금’ 등에 대한 각기 다른 해석으로 떠들썩하다. 대통령은 “하나의 공정이 또 다른 불공정을 부르는 상황”을 언급하며 어려움을 토로하기도 했다. 이렇듯 ‘공정’이라는 하나의 화두를 두고 각계각층이 충돌하고 있는 상황에서, 마이클 샌델 하버드대 교수가 『돈으로 살 수 없는 것들』 이후 8년 만에 쓴 신간 『공정하다는 착각』이 출간되었다. 

이 책은 『The Tyranny of Merit: What’s Become of the Common Good?』란 원제로 미국 현지에서 2020년 9월에 출간되어 뜨거운 반응을 얻고 있다. 직역하면 ‘능력주의의 폭정: 과연 무엇이 공동선을 만드나?’다. 샌델은 이 책을 통해 “우리가 ‘노력하면 성공할 수 있다’고 너무나도 당연히 생각해왔던, 개인의 능력을 우선시하고 보상해주는 능력주의 이상이 근본적으로 크게 잘못되어 있다”고 주장한다. 이러한 능력주의가 제대로 공정하게 작동하고 있는지, ‘공정함=정의’란 공식은 정말 맞는 건지 진지하게 되짚어본다.

                                    

2010년 이후, 한국에 ‘정의’ 열풍을 불러일으켰다. 27세에 최연소 하버드대학교 교수가 되었고, 29세에 자유주의 이론의 대가인 존 롤스의 정의론을 비판한 『자유주의와 정의의 한계』(1982)를 발표하면서 세계적으로 명성을 얻었다. 1980년부터 하버드대학교에서 정치철학을 가르치고 있으며, 그의 수업은 현재까지 20여 년 동안 학생들 사이에서 최고의 명강의로 손꼽힌다. 존 롤스 이후 정의 분야의 세계적 학자로 인정받는 그는 명실공히 이 

# 6 역행자 : 추천사 없음 [100]

In [16]:
url = 'http://www.yes24.com/Product/Goods/119120517'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/119120517?goodsSortNo=001022&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271455934'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



한층 더 강력한 치트키,  독해진 팩트 폭격!50주 연속 베스트셀러·40만 독자의 삶을 바꾼  『역행자』가 확장판으로 돌아왔다“절대로 읽지 마라! 죽을 때까지 순리자로 살고 싶다면!”  95퍼센트의 인간은 타고난 유전자와 본성의 꼭두각시로 살아간다. 이들은 평생 돈, 시간, 운명에게 속박되어, 평범함을 벗어나지 못하고 불행하게 사는 ‘순리자’다. 그러나 5퍼센트의 인간은 다르다. 이들은 타고난 유전자의 본성을 역행해 돈, 시간, 운명으로부터 완전한 자유를 얻는다. 본성을 거슬러 행복을 쟁취하는 이들이 바로 ‘역행자’다. 운명과 본능의 지배에서 벗어나 경제적 자유와 행복을 쟁취하는 라이프해킹의 비밀을 담은 책, 『역행자』가 2022년 출간된 지 1년 만에 확장판으로 다시 돌아왔다. ‘무자본 연쇄창업마’로서 30대 초반에 자동화 수익을 완성한 자청, 그가 10년 동안 시행착오를 겪으며 찾아낸 ‘역행자 7단계’ 모델을 담은 『역행자』는 2022년 출간 즉시 ‘전 서점 종합 베스트셀러 1위 달성’, ‘1년 만에 40만 부 판매’라는 대기록을 세우며 대한민국에 ‘역행자 신드롬’을 불러일으켰다. 『역행자』 확장판은 더욱 묵직해진 팩트 폭격과 뼈 때리는 사례들, 저자의 비밀 노하우인 ‘무자본 창업 7단계 공식’ 등을 수록하며 초판에 비해 무려 100페이지 가량의 분량이 추가되었다. 저자의 생생한 육성을 그대로 담았을 뿐만 아니라 『역행자』를 만나 완전히 다른 인생을 살고 있는 독자들의 생생한 경험담을 수록하여 독자들로 하여금 역행자가 되지 않고는 버틸 수 없게 만든다. 한층 더 강력해진 『역행자』 확장판만 있다면, 당신도 저자와 함께 ‘돈, 시간, 운명’으로부터 완전한 자유를 얻게 될 것이다.* 이 책은 2022년 출간된 『역행자』의 개정증보판입니다.                     

30대 초반에 경제적 자유를 이룬 사업가이자 ‘역행자’ 신드롬을 일으킨 베스트셀러 작가, 그리고 유튜브 크리에이터다.자청은 ‘자수성가 청년’의 줄임말로, 이미 30대 초반에 어떤 일

# 7 밑바닥부터 시작하는 딥러닝 : 추천사 없음 [600]

In [17]:
url = 'http://www.yes24.com/Product/Goods/34970929'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/34970929?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271541390'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



직접 구현하고 움직여보며 익히는 가장 쉬운 딥러닝 입문서 이 책은 라이브러리나 프레임워크에 의존하지 않고, 딥러닝의 핵심을 ‘밑바닥부터’ 직접 만들어보며 즐겁게 배울 수 있는 본격 딥러닝 입문서이다. 술술 읽힐 만큼 쉽게 설명하였고, 역전파처럼 어려운 내용은 ‘계산 그래프’ 기법으로 시각적으로 풀이했다. 무엇보다 작동하는 코드가 있어 직접 돌려보고 요리조리 수정해보면 어려운 이론도 명확하게 이해할 수 있다. 딥러닝에 새롭게 입문하려는 분과 기초를 다시금 정리하고 싶은 현업 연구자와 개발자에게 최고의 책이 될 것이다.                    

1984년 나가사키 현 쓰시마 태생. 도쿄공업대학교 공학부를 졸업하고 도쿄대학대학원 학제정보학부 석사 과정을 수료했다. 현재는 기업에서 컴퓨터 비전과 기계학습 관련 연구·개발에 매진하고 있다. 오라일리재팬에서 『밑바닥부터 시작하는 딥러닝』을 집필했으며, 『실천 파이썬 3』, 『컴퓨터 시스템의 이론과 구현』, 『실천 기계학습 시스템』 등을 번역했다.
1장 헬로 파이썬
1.1 파이썬이란?
1.2 파이썬 설치하기
__1.2.1 파이썬 버전
__1.2.2 사용하는 외부 라이브러리
__1.2.3 아나콘다 배포판
1.3 파이썬 인터프리터 
__1.3.1 산술 연산 
__1.3.2 자료형 
__1.3.3 변수 
__1.3.4 리스트 
__1.3.5 딕셔너리
__1.3.6 bool 
__1.3.7 if 문 
__1.3.8 for 문 
__1.3.9 함수 
1.4 파이썬 스크립트 파일 
__1.4.1 파일로 저장하기 
__1.4.2 클래스 
1.5 넘파이 
__1.5.1 넘파이 가져오기 
__1.5.2 넘파이 배열 생성하기 
__1.5.3 넘파이의 산술 연산 
__1.5.4 넘파이의 N차원 배열 
__1.5.5 브로드캐스트 
__1.5.6 원소 접근 
1.6 matplotlib 
__1.6.1 단순한 그래프 그리기 
__1.6.2 pyplot의 기능 
__1.6.3 이미지 표시하기 
1.7 정리 

2장 퍼셉트론
2.1 퍼

# 8 (윤성우의 열혈) C++ 프로그래밍 : 추천사 없음 [600]

In [18]:
url = 'http://www.yes24.com/Product/Goods/3816661'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/3816661?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271673830'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



2004년도에 출간된 윤성우 저자 「열혈강의 C++ 프로그래밍」의 개정판이다. C언어를 이해하고 있는 독자들을 대상으로 한 C++ 기본서로서, 초보자에게 적절한 설명과 예제를 통해서 C++ 학습에 재미를 더하고 있다. 개정판에서는 초판에는 없었던 내용들이 약 120 페이지 분량으로 추가하였으며, 예제도 시대의 흐름에 맞게 수정하였다. 총 4개의 파트로, 객체지향의 완성-전개-도입-C++로의 전환을 다루고 있다. 각 장이 끝날때 마다 연습문제를 수록하여 내용의 이해 정도를 스스로 확인해볼 수 있다. C언어 기반의 C++에 대한 기본적인 이해부터 클래스, 복사 생성자, 상속, 연산자 등의 세부 내용까지 다루어 C++프로그래밍에 대한 자신감을 갖게 한다.                     

벤처회사에서 개발자로 일하다가 IT분야의 집필과 강의로 처음 이름이 알려진 그는 2000년대 초반까지는 소프트웨어 아키텍트(Architect)로 일을 하다가, 2004년부터 지금까지 OpenGL-ES 그래픽스 라이브러리의 구현과 3D 가속 칩의 개발 및 크로노스 그룹(모바일 국제 표준화 컨소시엄)의 표준안에 관련된 일에 참여하였다. 또한 핸드폰용 DMB 칩의 개발에도 참여하였으며, 현재는 ㈜액시스소프트의 CTO로 있으면서 웹 기반 솔루션 개발에 관심을 갖고 있다. 시간이 날 때마다 조깅을 하며 다양한 구상을 하는 저자는, 방법론에 근거한 소프트웨어 설계 전문가가 모든 분야에서 활발히 활동하여 소프트웨어 발전에 기여해야 한다는 생각을 갖고 있다.프로그래밍을 공부한다면 모르는 사람이 없을 것 같은 베스트셀러 저자이다. 여전히 쉽게 설명하는 방법에 대해 고민하고 있으며 그 고민 결과를 바탕으로 책을 집필하고 있다. 프로그래머라는 직업에 충실하기위해 적지않은 기간동안 집필활동이 없었지만 그간 축적된 에너지를 바탕으로 다수의 책을 쓸 계획을가지고 집필에 임하고 있다.
Part 01 C++로의 전환
   Chapter 01  C언어 기반의 C++ 1
      01-1 printf와

# 9 아몬드 : 목차, 추천사 없음 [800]

In [19]:
url = 'http://www.yes24.com/Product/Goods/37300128'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/37300128?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=05&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271792911'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



공감 불능 사회, 차가움을 녹이는 아몬드
매혹적인 문체, 독특한 캐릭터, 속도감 넘치는 전개! 
“고통과 공감의 능력을 깨우치게 할 강력한 소설”

영화보다 강렬하고 드라마처럼 팽팽한, 완전히 새로운 소설이 나타났다. ‘감정을 느끼지 못하는’ 소년의 특별한 성장 이야기로, 첫 장부터 강렬한 사건으로 시작해 다음 페이지가 궁금해지게 만드는 흡입력 강한 작품이다. 또한 타인의 감정에 무감각해진 ‘공감 불능’인 이 시대에 큰 울림을 주는 소설로, 작품 속 인물들이 타인과 관계 맺고 슬픔에 공감하며 성장해 나가는 과정을 탁월하게 묘사했다. 영화처럼 펼쳐지는 극적인 사건과 매혹적인 문체로 독자를 단숨에 사로잡을 것이다. 『완득이』 『위저드 베이커리』를 잇는 제10회 창비청소년문학상 수상작. 

『아몬드』의 주인공 ‘윤재’는 감정을 느끼는 데 어려움을 겪는 독특한 캐릭터다. 다른 사람의 말과 행동의 이면을 읽어 내지 못하고 공포도 분노도 잘 느끼지 못하는 윤재는 ‘평범하게’ 살아가려고 가까스로 버텨 오고 있다. 엄마에게서 남이 웃으면 따라 웃고, 호의를 보이면 고맙다고 말하는 식의 ‘주입식’ 감정 교육을 받기도 한다. 세상을 곧이곧대로만 보는 아이, ‘괴물’이라고 손가락질받던 윤재는 어느 날 비극적인 사건을 맞아 가족을 잃게 되면서 이 세상에 홀로 남는다. 

그런데 모든 것을 잃었다고 생각하던 순간에 윤재 곁에 새로운 인연이 다가온다. 어두운 상처를 간직한 아이 ‘곤이’나 그와 반대로 맑은 감성을 지닌 아이 ‘도라’, 윤재를 돕고 싶어 하는 ‘심 박사’ 등이 그러한 인물들이다. 윤재와 이들 사이에서 펼쳐지는 이야기는 우리로 하여금 타인의 감정을 이해한다는 것이 얼마나 어려운지, 그럼에도 얼마나 소중한 일인지 다시 한번 생각해 보게 한다. 상실을 애도할 시간, 감정을 보듬을 여유를 잃어버린 채 살고 있는 독자들은 윤재를 응원하면서 자신의 마음 또한 되돌아볼 기회를 얻을 것이다. 윤재의 덤덤한 어조는 역설적으로 읽는 이의 가슴을 더욱 슬프게 저미며, 독자는 이 작품을 

# 10 우리가 빛의 속도로 갈 수 없다면 : 추천사 없음 [800]

In [20]:
url = 'http://www.yes24.com/Product/Goods/74261416'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/74261416?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271851170'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



우리 SF의 우아한 계보, 그 후지난겨울까지 바이오센서를 만드는 과학도였던 김초엽 작가는, 이제 소설을 쓴다. 「관내분실」로 한국과학문학상 중단편부문 대상을 받았다. 필명으로 낸 「우리가 빛의 속도로 갈 수 없다면」도 동시에 상을 받았다. ‘한국 SF의 우아한 계보’라 불리며 작품 활동을 시작한 김초엽 작가는 그 후, 더욱 도약했다. 자신만이 그려낼 수 있는 김초엽 특유의 작품세계를 보여주었다. 투명하고 아름답지만 순진하지만은 않은, 어디에도 없는 그러나 어딘가에 있을 것 같은, 근사한 세계를 손에 잡힐 듯 이야기에 담아냈다.다섯 개의 위성이 뜨는 곳에서도, 지지 않는 마음표제작 「우리가 빛의 속도로 갈 수 없다면」에는 매력적인 ‘할머니 과학자’가 주인공으로 등장한다. 인물을 통해 소설은 어째서 어떤 고통은 기꺼이 감내할 수 있는지, 생의 끝에서 최선을 다한다는 것은 무엇인지를 자꾸만 묻는 듯하다. 문학상 이후 김초엽의 작품들은 더욱 확장된 세계를 그려낸다. 작가의 고민과 질문도 더 단단해진듯하다. 다섯 개의 위성이 뜨는 행성에 홀로 남겨져 외계인과 조우하게 될지라도(「스펙트럼」), 고통 없는 유토피아에서 짐짓 모르는 것처럼 질문하지 않고 살아갈 수 있을 때에도(「순례자들은 왜 돌아오지 않는가」). 그럼에도 불구하고 나의 세계를, 우리의 세계를 알아야겠다고 용기 내는 마음, 우리의 사랑과 우정을 말하며 지지 않는 마음, 분투하는 태도가 김초엽의 소설에는 있다.소녀들의 영웅이 금메달리스트일 필요는 없다김초엽은 정상과 비정상, 성공과 실패, 주류와 비주류의 경계를 끊임없이 질문한다. 미션에 실패했다고 비난받는 우주인일지라도(「나의 우주 영웅에 관하여」), 어떤 소녀에게는 그의 존재 자체가 응원일 수 있다. 무엇이 성공이고, 무엇이 실패인가. 우주 미션에는 실패했지만, 소녀를 응원하는 일에 성공했다면 그 삶을 실패한 삶이라 할 수 있을까. 소녀들의 영웅이 금메달리스트일 필요는 없다. 경계에 선 소설가 김초엽은 고민과 질문을 쨍하게 빛나는 이야기로 들려준다. 그것

# 11 칵테일, 러브, 좀비 : 추천사 없음 [800]

In [21]:
url = 'http://www.yes24.com/Product/Goods/89913383'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/89913383?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271909377'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



안전가옥 쇼-트 시리즈의 두 번째 책으로, 조예은 작가의 단편집이다. 안전가옥 오리지널 시리즈의 첫 책 『뉴서울파크 젤리장수 대학살』에서 탄탄한 구성의 호러 스릴러를 선보였던 작가의 연출력은 단편집에서 더욱 다양한 색채로 빛을 발한다. 미묘하지만 분명한 폭력을 감내해 왔던 여성 빌런의 탄생을 그린 「초대」, 물귀신과 숲귀신 사이의 사랑스러운 이끌림을 담은 「습지의 사랑」, 블랙 유머를 통해 가부장제의 이면을 들여다보는 오컬트 좀비물 「칵테일, 러브, 좀비」, 제2회 황금가지 타임리프 공모전에서 우수상을 차지한 「오버랩 나이프, 나이프」 등 네 작품을 수록하였다. 

                                    

제2회 황금가지 타임리프 공모전에서 『오버랩 나이프, 나이프』로 우수상을, 제4회 교보문고 스토리 공모전에서 『시프트』로 대상을 수상했으며 최근작으로는 안전가옥의 첫 번째 장편소설 『뉴서울파크 젤리장수 대학살』, 『스노볼 드라이브』, 소설집 『칵테일, 러브, 좀비』가 있다. 좋은 이야기에 대해 고민하며 작품 활동을 계속하는 중이다.
초대
습지의 사랑
칵테일, 러브, 좀비 
오버랩 나이프, 나이프

작품 후기
프로듀서의 말
이토록 생생한 어둠어떤 감정은 곧잘 무시당한다. 여성이라서, 자식이라서, 부유하지 못해서, 남들과 어울리지 못해서 겪는 어둡고 축축한 마음이 그렇다. 괴로움을 호소했다가는 너무 예민하다는 평가를 받는다. 그런 문제는 별것 아니라고들 한다. 조예은 작가는 『칵테일, 러브, 좀비』 속 모든 작품에서 홀대받는 감정들을 생생하게 끄집어내며 반기를 든다. 그러한 감정들에는 분명한 실체가 있으며 그 주인에게 구체적인 고통을 안긴다. 허리가 길다고, 이마가 좁다고, 저번에 입은 옷은 영 별로였다고 쉽게 평가하는 남자친구를 향해 바로 전하지 못한 말들은 가시가 되어 목구멍을 찌른다(「초대」). 수십 년 인생을 남편 뒷바라지에 바친 아내는 좀비로 변한 남편을 보며 “저 막돼먹은 인간 없이 사는 게” 무섭다며 울먹인다(「칵테일,

# 13 어서 오세요, 휴남동 서점입니다 [800]

In [22]:
url = 'http://www.yes24.com/Product/Goods/106211628'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/106211628?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685271959762'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



“받아들여지고 있다는 느낌이이렇게 슬프고도 좋을 줄이야!”브런치북 전자책 출판 프로젝트 수상작! 독자 요청 쇄도로 전격 종이책 출간“책과 서점을 통해 새로운 관계를 만들고, 스스로 일어서기 위해 애쓰는 사람들의 이야기가 깊이 있게 펼쳐진다.”(소설가 김금희 심사평)서울 어디에나 있을 것 같은 동네의 후미진 골목길. 오가는 사람도 많지 않은 가정집들 사이에 평범한 동네 서점 하나가 들어선다. 바로 휴남동 서점! 슬픈 사연을 갖고 있는 사람처럼 얼굴에 아무런 의욕도 보이지 않는 서점 주인 영주는 처음 몇 달간은 자신이 손님인 듯 일은 하지 않고 가만히 앉아 책만 읽는다. 그렇게 잃어버린 것들을 하나둘 되찾는 기분으로 하루하루를 보내다 보니 소진되고 텅 빈 것만 같았던 내면의 느낌이 서서히 사라진다. 그러다 어느 순간 깨닫는다. 자신이 꽤 건강해졌다는 사실을. 그 순간부터 휴남동 서점은 완전히 새로운 공간이 된다. 사람이 모이고 감정이 모이고 저마다의 이야기가 모이는 공간으로. 바리스타 민준, 로스팅 업체 대표 지미, 작가 승우, 단골손님 정서, 사는 게 재미없는 고등학생 민철과 그의 엄마 희주 등 크고 작은 상처와 희망을 가진 사람들이 휴남동 서점이라는 공간을 안식처로 삼아 함께 살아가는 법을 배운다. 『어서 오세요, 휴남동 서점입니다』는 우리가 잃어버린 채 살고 있지만 사는 데 꼭 필요한 것들이 가득한 책이다. 배려와 친절, 거리를 지킬 줄 아는 사람들끼리의 우정과 느슨한 연대, 진솔하고 깊이 있는 대화 등. 출간 즉시 전자책 TOP 10 베스트셀러에 오르며 수많은 독자의 찬사를 받은 소설이 독자들의 강력한 요청으로 마침내 종이책으로 다시 태어났다.                     

대학에서 컴퓨터공학을 전공하고 LG전자에서 소프트웨어 개발자로 일했다. 몇 번의 입사와 퇴사를 반복하면서도 매일 읽고 쓰는 사람으로서의 정체성은 잃지 않고 있다. 지은 책으로 『매일 읽겠습니다』, 『난생처음 킥복싱』, 『이 정도 거리가 딱 좋다』 등이 있다.
서점은 어떤 모

# 14 (2022 시나공) 컴퓨터활용능력 : 추천사 없음 [600]

In [23]:
url = 'http://www.yes24.com/Product/Goods/104245824'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/104245824?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=05&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272023395'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



[659개 필드 중 527개 필드 토막강의] 합격에 필요한 내용만 골라 듣는 시나공식 맞춤 동영상 강의를 제공하여 수험생의 부담을 줄이고 독학을 가능하게 하였다. 『2022 시나공 컴퓨터활용능력 1급 필기』는 실제 시험의 출제 경향을 친절하게 짚어주고, 자주 시험에 나온 내용은 확실히 강조하여 혼자서도 충분히 공부할 수 있게 도와준다. 지금까지 출제된 모든 기출문제를 통계적으로 분석하여 시험에 나오는 것만 골라 168개의 섹션, 659개의 필드로 정리하였다. 섹션은 시험에 출제된 비율에 따라 A, B, C, D 등급을 지정하고, 각 필드에는 시험에 나온 출제연도를 표기하여 수험생이 더 효율적으로 공부할 수 있도록 하였다.[마지막 시험장까지 책임지는 필수암기집] 최근 7년간 출제된 기출문제 1080제를 압축한 필수 암기 206선을 모아 기출문제를 풀지 못했더라도 이것만 읽어보면 학습을 마무리할 수 있다. [기억상자&기출 CBT] ‘기억상자’ 암기 프로그램으로 반복 학습을 통해 틀린 문제는 틀리지 않을 때까지, 맞혔던 문제는 안전하게 머릿속에 담아 시험장에 가지고 갈 수 있도록 해주고, 실제 시험장과 똑같은 기출문제 CBT로 마무리 학습 및 시험 전 현장 감각을 키울 수 있다(www.membox.co.kr).                    

IT 서적을 기획하고 집필하는 출판 기획 전문 집단으로, 2003년부터 길벗출판사의 IT 수험서인 『시험에 나오는 것만 공부한다!』 시리즈를 기획부터 집필 및 편집까지 총괄하고 있다. 20여 년간 자격증 취득에 관한 교육, 연구, 집필에 몰두해 온 강윤석 실장을 중심으로 IT 자격증 시험의 분야별 전문가들이 모여 국내 IT 수험서의 수준을 한 단계 높이기 위한 다양한 연구와 집필 활동에 전념하고 있다.
[1과목] 컴퓨터 일반 

1장 한글 Windows의 기본
2장 한글 Windows 10의 고급 기능
3장 컴퓨터 시스템의 개요
4장 컴퓨터 하드웨어
5장 컴퓨터 소프트웨어
6장 인터넷 활용
7장 멀티미디어 활용
8장 

# 15 천관사복 : 목차/출판사 리뷰/추천사 없음 [800]

In [24]:
url = 'http://www.yes24.com/Product/Goods/109581474'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/109581474?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272067079'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



만인의 사랑을 받는 선락국의 금지옥엽 태자 사련.누구보다 빠르게 선경에 오르나한순간에 법력이 봉인된 채 인간 세상에 떨어진다.그렇게 신선과 인간의 경지를 수차례 오가던 사련은‘공덕’을 모으기 위해 인간 세상으로 내려가게 되고……어느 날 고물을 줍고 돌아오는 길에신비한 소년 ‘삼랑’을 만나이 범상치 않은 소년과 함께 지내기로 한다.‘공덕’을 모으기 위해 분주한 가운데 벌어진 괴이한 사건,사련과 삼랑, 그리고 선계에서 온 두 어린 신관,넷은 사건을 해결하며 선계의 가려진 어둠에 한 발자국씩 다가가게 되는데……!                    

미신을 믿는 소녀이자 유명한 이모티콘 블로거. 『마도조사』, 『인사반파자구계통』, 『인사반파자구계통』 시리즈 등을 썼다. “미식계의 이단아로 사진을 찍을 때 손을 떤다. 타자가 느리고 기분에 따라 글을 쓴다. ……사실은 모두 거짓말이다. 오후에 차 한 잔을 마시며 먼 곳을 바라보면서 아끼는 노트에 시를 쓰는 것을 좋아한다. ……아니, 이게 더 거짓말이다. 그냥 사실 나는 그저 글을 쓰는 사람이다.”
없음
없음
없음
이북으로 전권 구매했었는데 정말 좋아하는 책이라 실물책으로도 구매했습니다. 책 표지가 굉장히 멋스럽고 예쁘게 나왔습니다. 화성이미지의 나비 문양이 좋았습니다. 1권에서 정식으로 화성이 등장하는것은 중반부입니다. 초반부에는 잠깐 얼굴만 내미는정도입니다.그러나주인공인 사련과 도와주는 신관 2명이 매력있어서 화성의 등장여부와 상관없이 책은 내내 정...더보기
이북으로 전권 구매했었는데 정말 좋아하는 책이라 실물책으로도 구매했습니다. 책 표지가 굉장히 멋스럽고 예쁘게 나왔습니다. 화성이미지의 나비 문양이 좋았습니다. 1권에서 정식으로 화성이 등장하는것은 중반부입니다. 초반부에는 잠깐 얼굴만 내미는정도입니다.그러나 주인공인 사련과 도와주는 신관 2명이 매력있어서 화성의 등장여부와 상관없이 책은 내내 정말 재밌게 읽었습니다. 1권은 사막이야기 중반에서 끝맺었네요.
묵향동후 작가의 천관사복 1권입니다.

마도조사, 인

# 16 미드나잇 라이브러리 : 목차, 추천사 없음 [800]

In [25]:
url = 'http://www.yes24.com/Product/Goods/99534783'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/99534783?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272127305'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



밤 12시, 죽기 바로 전에만 열리는 마법의 도서관에서
인생의 두 번째 기회를 드립니다
국내 주요 서점 종합 베스트셀러 1위, 2021년 올해의 책 선정
[뉴욕타임스] 베스트셀러 1위, 영국 아마존 종합 베스트셀러 1위

매트 헤이그의 소설 『미드나잇 라이브러리』는 2021년 4월 출간 이후 10개월 만에 30만 독자의 마음을 사로잡으며 눈물과 웃음, 가슴 뭉클한 감동을 선사하고 있다. 죽기로 결심한 주인공 ‘노라 시드’가 삶과 죽음 사이에 존재하는 미스터리한 도서관 ‘미드나잇 라이브러리’에서 눈을 뜨며 인생의 두 번째 기회를 얻는 이 소설은, 노라의 가장 완벽한 삶을 찾는 모험을 따라가며 ‘살아 있음’과 ‘살아가는 것’에 대한 깊은 통찰을 보여준다.

“책을 덮는 순간 내일이 기대된다!” “내 인생의 반려책을 만났다” “다시 살아봐야겠다는 힘을 얻었다” 등 이 책을 읽은 독자들의 뜨거운 반응에 힘입어 주요 서점에서 2021 올해의 책으로 선정되었고, 그 열기는 여전히 계속되고 있다. 타인의 삶에 대한 공감, 자신의 삶에 대한 긍정, 살아 있는 것 자체가 살아야 할 이유라는 깊은 통찰력까지 매트 헤이그 작가가 보내는 공감과 위로의 메시지는 세대와 성별을 아우르며 전 세계 독자들의 마음을 사로잡았다. [뉴욕타임스] 베스트셀러 1위, 영국 아마존 종합 베스트셀러 1위를 기록했고, 미국 아마존과 [뉴욕타임스] 60주 이상, 독일 [슈피겔]에서도 38주 연속 장기 베스트셀러에 올랐다. 현재 덴마크, 이탈리아, 스페인, 루마니아, 타이완, 이란 등 세계 각지에서 출간되며 글로벌한 인기를 누리고 있다. 




                                    

영국 요크셔 출신의 동화작가 겸 소설가다. 2004년에 출간한 첫 소설 『영국의 마지막 가족』을 비롯하여 10편의 성인 대상 작품과 12편의 동화 및 청소년 소설을 발표해왔다. 20대 초반에 정신적 위기를 맞은 그는 절벽 끝에 서서 스스로 생을 마감하려던 순간, 자신의 우울증과 불안장애를 깨닫

# 17 (김대식의) 인간 vs 기계 : 추천사 없음 [100]

In [26]:
url = 'http://www.yes24.com/Product/Goods/25258568'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/25258568?goodsSortNo=001014&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272171133'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



인간의 뇌와 기계의 뇌, 모두에 정통한김대식 교수만이 이야기할 수 있는 인공지능 강의	카이스트 김대식 교수는 전작 『김대식의 빅퀘스천』을 통해 그만의 철학적 깊이와 인문학적 시선들을 드러낸 바 있다. [김대식의 북스토리], [김대식의 브레인 스토리]등을 연재하며 과학을 대하는 인문학적 성찰의 중요성을 전하는 인물이기도 하다. [장영실쇼], [창의 인재 프로젝트 생각의 집]등 각종 매체·방송·저서로 전 연령층의 독자에게 친숙하고 대중적인 저자다. 그의 강연은 쉽고, 위트 있으며, 흥미롭고 재미있다. 막연했던 과학적 궁금증을 콕 집어 긁어주면서도 개인과 사회가 고민해야 할 화제를 던져준다.	김대식 교수는 막스-플랑크 뇌과학 연구소에서 뇌과학 박사학위를 받고 MIT에서 뇌인지과학을 공부했다. 그리고 현재 카이스트에서 인공지능을 연구하고 있다. 알파고의 충격에서 허우적거리던 2016년 3월 17일, 청와대는 지능정보사회 민관 합동 간담회를 열었고 인공지능 연구자 대표로 김대식 교수가 초청되었다. 즉, 대한민국 인공지능 분야 최전방에 있는 인물이다. 	전통적인 인공지능과 현재의 인공지능, 두 인공지능의 거의 유일한 공통점은 인간의 지능을 모방했다는 점이다. 전통적인 인공지능은 인간이 이해한 지능을 기계에게 구현하려 했으며, 현재의 인공지능은 인간의 학습방법과 인간의 신경세포층의 구성을 모방하여 기계에 구현하고 있다. 결국 인간의 뇌를 이해하는 것이야말로 인공지능을 이해하게 되는 시작점이다. 	김대식 교수는 인간의  뇌와 기계의 뇌 두 가지를 깊이 탐구한 유일무이한 사람이다. 따라서 김대식 교수만이 할 수 있는 인공지능에 대한 이야기가 있다. 인간의 지능을 이해하는 것은 인공지능의 능력과 인공지능의 발달을 예측하는 데 필수적이다. 김대식 교수의 뇌과학 강연이 곁들여진 인공지능 강의는 여타 다른 이야기보다 이해하기 쉽고 명료하면서도 명쾌하다.                    

뇌과학자. KAIST 전기및전자공학부 교수. 주요 연구 분야는 뇌과학, 뇌공학, 인공지능으

# 18 (이기적) 컴퓨터활용능력 : 추천사 없음 [600]

In [27]:
url = 'http://www.yes24.com/Product/Goods/111102308'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/111102308?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272228687'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



현 출제 기준을 완벽하게 반영한 본 도서는 필기시험을 준비하는 수험생을 위한 기본서이다. 다년간의 출제 경향을 철저히 분석하여 집필한 이론서 2권과 상시 기출문제를 삽입한 기출공략집으로 구성하였다. 1권/2권에서는 무료 동영상 강의와 함께 이론을 세세하게 공부할 수 있으며, 파트와 섹션마다 출제 빈도를 정리하고 시험에 출제된 연도를 표기하여 수험생이 더욱 효율적으로 학습할 수 있도록 하였다. 또한 이론 학습 후에는 기출문제와 예상 문제로 그때그때 학습한 내용을 바로 확인할 수 있도록 하였으며, 난이도가 높은 문제는 무료 동영상 강의를 추가하여 확실히 이해하고 넘어갈 수 있도록 하였다. 3권 기출공략집에서는 자주 출제되는 기출문제 120선을 통해 최신 출제 경향을 파악하고, 2021~2022년 상시 기출문제를 삽입하여 최신 상시 시험 유형에 대비할 수 있도록 하였다. 이처럼 『2023 이기적 컴퓨터활용능력 2급 필기 기본서』는 단 한 권으로 합격할 수 있도록 구성된 최적의 학습서이다.                    

2008년부터 이기적 컴퓨터활용능력 필기 도서를 담당하고 있다. 이패스코리아에서 컴퓨터활용능력 강의를 하고 있으며, 강원교육과학정보원 SME, 고용노동부 직훈 교사를 역임하고 있다. 2019 교육부장관 표창장을 수상했으며, 컴퓨터활용능력 필기, 정보처리기사 필기, e-Test Professionals MS-EXCEL 등을 집필하였다.
출제 경향 분석
맞춤 학습 플랜
차례
이기적 합격 서비스
이 책의 구성
시험 안내
자주 질문하는 Q&A

[1권] Part 1 컴퓨터 일반 
Chapter 1. 운영체제 사용
Chapter 2. 컴퓨터 시스템 설정 변경
Chapter 3. 컴퓨터 시스템 관리
Chapter 4. 인터넷 자료 활용
Chapter 5. 컴퓨터 시스템 보호
정답 및 해설
쏙쏙 찾아보는 INDEX(1권, 2권)

[2권] Part 2 스프레드시트 일반 
Chapter 1. 스프레드시트 개요
Chapter 2. 데이터 입력 및 편집 
C

# 19 컴퓨터 네트워킹 : 책소개/저자소개/목차/추천사/리뷰 없음 [600]

In [28]:
url = 'http://www.yes24.com/Product/Goods/112228953'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

print('회원리뷰 없음')

없음
저서로 『컴퓨터 네트워킹 : 하향식 접근』이 있다.
없음
없음
없음
회원리뷰 없음


# 20 (시나공) 컴퓨터활용능력 1급 실기 : 추천사 없음 [600]

In [29]:
url = 'http://www.yes24.com/Product/Goods/94183650'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/94183650?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=05&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272468367'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



『2021 시나공 컴퓨터활용능력 1급 실기』는 엑셀, 액세스의 방대한 기능 중 시험에 꼭 나오는 엑셀 기능 18가지, 액세스 기능 8가지로 압축하고, 쉽게 익힐 수 있도록 따라하기 형식으로 구성하였다. 각 섹션마다 출제된 비율에 따라 A, B, C, D 등급을 지정하여 중요한 내용을 먼저 볼 수 있다. 자주 틀리는 계산문제는 컴퓨터 없이도 틈틈이 풀어볼 수 있도록 별도의 책으로 엮어 실기 시험에 꼭 나오는 함수 사전, 수식 만드는 요령, 수식 이해하는 방법 등 계산문제 풀이에 대한 체계적인 학습전략을 수록했다.
                                    

IT 서적을 기획하고 집필하는 출판 기획 전문 집단으로, 2003년부터 길벗출판사의 IT 수험서인 『시험에 나오는 것만 공부한다!』 시리즈를 기획부터 집필 및 편집까지 총괄하고 있다. 20여 년간 자격증 취득에 관한 교육, 연구, 집필에 몰두해 온 강윤석 실장을 중심으로 IT 자격증 시험의 분야별 전문가들이 모여 국내 IT 수험서의 수준을 한 단계 높이기 위한 다양한 연구와 집필 활동에 전념하고 있다.
[1권] 스프레드시트 실무

[1부] 기본 편
준비운동
1. 스프레드시트 편, 이렇게 준비하세요.
2. 시험 접수부터 자격증 받기까지 한눈에 살펴볼까요?
3. 한눈에 보는 컴퓨터활용능력 1급 실기 시험 절차
4. 컴퓨터활용능력 시험, 이것이 궁금하다! - Q&A

1장 기본작업
Section 01 셀 서식 및 시트 보호 
Section 02 조건부 서식/중복된 항목 제거
Section 03 자동 필터/고급 필터 
Section 04 페이지 레이아웃/통합 문서 보기

2장 계산작업 
Section 05 논리 함수, 찾기 함수, 텍스트 함수, 정보 함수
Section 06 수학/삼각 함수, 통계 함수, 재무 함수
Section 07 데이터베이스 함수, 날짜/시간 함수, 사용자 정의 함수
Section 08 배열 수식

3장 분석작업 
Section 09 데이터 유효성 검사
Section 

# 21 다정한 것이 살아남는다 [500]

In [30]:
url = 'http://www.yes24.com/Product/Goods/102836095'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/102836095?goodsSortNo=001020&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272522536'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



저자 친필 사인 인쇄본국내 주요 매체 선정 2021년·2022년 올해의 책책씨앗 선정 2022년 최고의 책전국도서관대회 선정 2022년 사서 베스트 도서제62회 한국출판문화상 번역 부문 최종 후보작 김영하 북클럽이 선정한 최초의 과학책문재인 제19대 대한민국 대통령 추천 도서 최재천, 강양구, 이원영, 은유, 정세랑, 하미나, 김겨울, 서미란, 엄지혜, 위다혜, 김경영 추천유독 작가들로부터 사랑받은, 작가들의 영감을 불러일으킨 책 ‘가장 간절한 순간’에 찾아온 ‘21세기 다윈의 계승자’가 쓴 ‘완벽한 책’!정세랑 작가가 쓴 추천의 말처럼 “어떤 책은 그 책이 가장 간절한 순간을 골라 찾아온다”. 『다정한 것이 살아남는다』 한국어판은 2021년 7월 출간 이후 한국 독자들에게 뜨거운 사랑을 받았으며, 누적 판매 10만 부를 넘어서며 전 세계에서 한국에서 가장 많이 판매되었다. 2022년 가을, 한국을 방문한 브라이언 헤어 박사는 이를 ‘놀라운 사건’이라 말하며 다정한 한국 독자들에게 특별한 감사의 인사를 전하기도 했다. 출간된 지 두 해가 되어가지만 이 책에 대한 독자들의 사랑은 식지 않고 있으며, ‘다정한 것이 살아남는다’는 한국 사회에서 희망과 긍정의 메시지로 자리 매김하고 있다. ‘10만 부 기념 스페셜 에디션’에는 저자들의 친필 사인과 친필 메시지를 담았다. 박연미 디자이너는 엄유정 작가의 새로운 그림으로 ‘진화와 번성에 성공한 다정한 생명체’인 초록 식물의 메시지를 구현해냈다. “한국어판 오리지널 버전에서는 사람들의 다정한 모습을 보여주었다면 이번 스페셜 에디션에서는 그 범위를 좀 더 넓혀 진화와 번성에 성공한 다정한 생명체 자연의 모습을 담았다. 엄유정 작가가 그려낸 초록의 둥근 잎과 교차하는 식물의 두 줄기가 서로 다정하게 인사를 나누는 듯하다. 그 인사에서 위로를 받는다. 같이 뻗어 나가는 식물의 가지처럼 우리도 다정하게 함께 나아가길 바라며.”_박연미, 디자이너의 말                    

듀크대학교의 진화인류학, 심리학, 신경

# 22 미움받을 용기 [100]

In [31]:
url = 'http://www.yes24.com/Product/Goods/116599423'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/116599423?goodsSortNo=001014&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272598495'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



200만 독자가 선택한 우리 시대의 새로운 고전!- 51주 연속, 역대 최장기간 베스트셀러 1위!- 전 세계 1000만 부 돌파한 글로벌 베스트셀러!모든 고민은 인간관계에서 비롯된다.타인에게 미움받는 것을 두려워하지 마라. 모든 것은 용기의 문제다.사람은 누구나 변화를 원한다. 지금보다 더 자유로운 삶, 지금보다 더 성공적인 삶, 지금보다 더 행복한 삶. 하지만 우리는 쉽게 핑계를 대고, 쉽게 포기한다. 지금 자신의 인생을 되돌아보자. “내게 조금 더 능력이 있었더라면”, “좀 더 부잣집에서 태어났더라면”, “참고 견디면 언젠가 나아지겠지”라며 환경을 탓하거나 현실을 외면하지는 않았는지 말이다. 이런 우리에게 “인간은 변할 수 있고, 누구나 행복해질 수 있다”라고 단언하는 철학자가 있다. 그간 상식으로 일컬어지던 프로이트의 ‘원인론’을 정면으로 부정하며, 자유도 행복도 모두 ‘용기’의 문제일 뿐 환경이나 능력의 문제가 아님을 보여준 알프레드 아들러(Alfred Adler)다. 2014년 출간돼 51주 연속 베스트셀러 1위를 기록하며 대한민국에 아들러 열풍을 일으킨 책 『미움받을 용기』가 국내 200만 부 판매를 기념하며 리커버 에디션으로 독자들을 찾아왔다. 이번 리커버는 8년간 변함없는 사랑을 보내준 한국 독자들에게 보내는 두 저자의 친서와 함께, 세련된 디자인과 따뜻한 삽화로 새 옷을 입었다. 오랫동안 아들러를 연구해 온 철학자 기시미 이치로의 혜안과 일본의 대표적인 스토리텔링 작가 고가 후미타케의 필력이 돋보이는 『미움받을 용기』는 2022년 현재 40개국 이상에서 번역 출간되었으며, 전 세계 누적 판매 1000만 부를 돌파하는 경이적인 기록을 세우기도 했다. 프로이트, 융과 함께 ‘심리학의 3대 거장’으로 불리는 아들러는 인간은 능력이나 환경, 과거의 트라우마와 관계없이 얼마든지 변할 수 있는 존재이며, 이를 위해서는 현재의 나를 있는 그대로 받아들이고 눈앞에 놓인 문제를 직시할 ‘용기’가 필요하다고 말한다. 이 책은 자유로워질 용기, 평범해질 용기 

<미움받을 용기 - 기시미 이치로, 고가 후미타케 저>친구한테 선물했어요.친구가 많이 힘들어하고, 싫은소리 못하고… 너무 순하고 착한 친구라서 선물하고 싶었네요.친구가 미움받은 용기를 냈으면 좋겠습니다.저 한테 역시 물론 필요한 내용이라 저 역시 사보려고 합니다.미움받은 용기란 정녕 뭘까, 요즘 시대를 사는 많은 사람들한테 필요한 책이 아닐까...더보기
<미움받을 용기 - 기시미 이치로, 고가 후미타케 저>친구한테 선물했어요.친구가 많이 힘들어하고, 싫은소리 못하고… 너무 순하고 착한 친구라서 선물하고 싶었네요.친구가 미움받은 용기를 냈으면 좋겠습니다.저 한테 역시 물론 필요한 내용이라 저 역시 사보려고 합니다.미움받은 용기란 정녕 뭘까, 요즘 시대를 사는 많은 사람들한테 필요한 책이 아닐까 싶습니다.
미움받을 용기 - 북 클러버



모든 사람이라면 당연히 누구에게나 인정 받고 싶어하고 또 이쁨받고 싶어할것이다.

이 책을 그런 사람들의 욕심 이면에 실패할것같은 두려움때문에 용기내지 못하고

나는 행복하지 않을 수 밖에 없는 이유를 몇가지씩 만들어 방어기제로 만들어 쓰는 것을

아들러 심리학을 바탕으로 쉽게 대화 형식으로 풀어내었다.



처음에는 유...더보기
미움받을 용기 - 북 클러버모든 사람이라면 당연히 누구에게나 인정 받고 싶어하고 또 이쁨받고 싶어할것이다.이 책을 그런 사람들의 욕심 이면에 실패할것같은 두려움때문에 용기내지 못하고나는 행복하지 않을 수 밖에 없는 이유를 몇가지씩 만들어 방어기제로 만들어 쓰는 것을아들러 심리학을 바탕으로 쉽게 대화 형식으로 풀어내었다.처음에는 유명한 책이니 한번 읽어보자 싶었는데읽어가다보니 베스트 셀러는 역시 이유가 있구나! 싶었다.사실 나 역시도 누구에게 변명의 여지로 '그렇게 할수밖에 없다'는 억지스러운 이유를 만들어낸것은 아니였나 싶다.
인간관계로 인한 스트레스를 덜 받기 위해

나 스스로선택과 집중을 위해 끊어내는 중이다.

어렸을 적에는미련하게도 그 관계가 끊어질까 조마했고 끊어지지 않게하기 위해

희생하고

# 23 법무사 5개년 기출문제해설 : 추천사 없음 [300]

In [32]:
url = 'http://www.yes24.com/Product/Goods/115257376'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/115257376?goodsSortNo=001001&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=99&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272658525'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



2023 법무사 1차시험 5개년 기출문제해설의 특징!

1. 모든 지문에 최대한 정확하고 상세한 해설을 수록하여 다른 교재를 찾아보아야 하는 수고로움 없이 본서만으로 충분히 학습할 수 있도록 하였다.

2. 각 지문마다 OX표시를 하여 지문별 개별학습이 가능하도록 하였고, 특히 틀린 지문의 해설에는 주요부분에 밑줄을 그어 그 내용을 한눈에 확인할 수 있도록 하였다.

3. 최신 법령·예규·판례·선례 및 실무제요에 근거하여 해설하였으며, 개정사항이 적용되어야 할 문제는 이를 반영하고 [기출수정]으로 표시하였다.

4. 법원실무제요 및 법원공무원교육원 교재와 같은 실무서의 내용이 그대로 지문으로 출제되는 과목의 해설에는 해당 내용을 직접 인용하고 별도로 표시하였으며, 특히 법원실무제요 민사집행의 경우에는 2020 최신 개정판의 내용을 반영하였다.

                                    

『법무사 1차시험 5개년 기출문제해설』의 저자이다.
『1권』

PART 1 헌 법

제1편 헌법총론
제2편 기본권론
제3편 통치구조론

PART 2 상 법

제1편 상법총칙
제2편 상행위법
제3편 회사법
제4편 어음법·수표법
제5편 보험법
제6편 해상법·항공운송법

『2권』

PART 3 민 법

제1편 민법총칙
제2편 물권법
제3편 채권총론
제4편 채권각론
제5편 친족·상속법

PART 4 가족관계의 등록 등에 관한 법률

제1편 총 설
제2편 등록사무처리절차
제3편 국제등록사무
제4편 등록부의 정정
제5편 가족관계등록비송
제6편 가족관계등록사무의 처분에 대한 불복
제7편 벌칙·과태료

『3권』

PART 5 민사집행법

제1편 총 론
제2편 강제집행
제3편 담보권 실행 등을 위한 경매
제4편 보전처분

PART 6 상업등기법 및 비송사건절차법

제1편 상업등기 총론
제2편 상업등기 각론
제3편 법인등기
제4편 부부재산약정의 등기
제5편 비송사건절차법

『4권』

PART 7 부동산등기법

제1편 총 론
제2편 각종 부동산등기절차

PART 8 공

# 24 (IFRS) 중급회계 : 추천사, 리뷰 없음 [300]

In [33]:
url = 'http://www.yes24.com/Product/Goods/108016770'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

print('회원리뷰 없음')



이번에 출간하는 제6판 IFRS 중급회계 상권에서는 제3장 '재무제표 표시'에서 회계기준 개정에 따라 ‘중요한 회계정책 정보의 공시’에 대하여 설명하였다. 그리고 제4장 '고객과의 계약에서 생기는 수익'에서 계약에 유의적인 금융요소가 있는 경우 관련 설명을 개선하고, 선불판매의 금융요소에 대한 설명을 보완하였다. 또한, 재매입약정과 관련하여 제조기업에 흔히 발생하는 사급거래에 대한 서술을 추가하였습니다. 그리고, 제5장 '재고자산'에서는 새로 발표된 해석에 따라 손실부담계약을 판단함에 있어 고정제조간접비 배부의 기준이 되는 조업도에 대해 설명하였다. 제6장 '유형자산'에서는 유형자산 취득시 매입한 국채 공채 등 관련 회계처리 설명을 추가하였다.이 책의 서술 목표는 가장 친절한 중급회계다. 중급회계에서 다루는 주제들은 광범위한데 한정된 페이지 속에서 이러한 목표를 달성하기 위해 모든 주제들을 간결하게 설명하기 위해 노력하였다. 그리고 기업회계기준서가 영어를 직역하여 한국어임에도 이해되기 어려운 점에 대한 독자의 입장을 고려하여 쉬운 말로 설명하기 위해 노력하였다. 더불어 이 책은 국내외에 출판된 모든 중급회계 책과 비교하여 각 장을 구성하는 주제들이 빠진 것이 없도록 주의하였다. 또한 본문의 CASE들은 어떤 문제와 비교해도 경쟁력 있는 수준으로 개발하였다. 가장 친절한 중급회계를 목표로 저술된 본 저서를 통해 독자들의 고민하는 시간이 줄어들 것이다.                    

서울시립대학교 경영대학 교수 연세대학교 경영학 박사 공인회계사, 세무사 전) 안진회계법인 감사본부
제1장 재무회계와 회계제도 	
제2장 재무보고 개념체계 및 현재가치
제3장 재무제표 표시 	
제4장 고객과의 계약에서 생기는 수익
제5장 재고자산	
제6장 유형자산
제7장 차입원가		
제8장 투자부동산 및 농림어업
제9장 무형자산	
제10장 자산손상 및 매각예정비유동자산
제11장 충당부채와 우발부채	
제12장 자본
이 책은 학계와 회계법인에서 다양하게 연구하고 실무를 경험한 배

# 25 (시나공 기출문제집) 컴퓨터활용능력 1급 : 추천사, 리뷰 없음 [600]

In [34]:
url = 'http://www.yes24.com/Product/Goods/105311235'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

print('회원리뷰 없음')



초단타 합격 전략!최근 6년간 출제된 최신 기출문제 15회분을 수록하고 문제마다 시험에 출제된 비율에 따라 A, B, C, D 등급을 지정하여 중요도를 표시했다. 개념과 함께 더 공부해야 할 문제, 문제와 지문을 외워야 할 문제, 답만 기억하고 넘어가라 문제들을 전문가가 꼼꼼히 알려준다. 수록된 기출문제는 문제만 이해해도 합격할 수 있도록 왜 답이 되는지 명쾌하게 알려주고, 새로운 문제 유형에도 대비할 수 있도록 설명한다.                    

IT 서적을 기획하고 집필하는 출판 기획 전문 집단으로, 2003년부터 길벗출판사의 IT 수험서인 『시험에 나오는 것만 공부한다!』 시리즈를 기획부터 집필 및 편집까지 총괄하고 있다. 20여 년간 자격증 취득에 관한 교육, 연구, 집필에 몰두해 온 강윤석 실장을 중심으로 IT 자격증 시험의 분야별 전문가들이 모여 국내 IT 수험서의 수준을 한 단계 높이기 위한 다양한 연구와 집필 활동에 전념하고 있다.
기출문제 & 전문가의 조언

01회 2021년 상시01 기출문제
02회 2021년 상시02 기출문제 
03회 2021년 상시03 기출문제 
04회 2021년 상시04 기출문제 
05회 2020년 2회 기출문제 
06회 2020년 1회 기출문제
07회 2019년 2회 기출문제
08회 2019년 1회 기출문제
09회 2018년 2회 기출문제
10회 2018년 1회 기출문제
11회 2017년 2회 기출문제 
12회 2017년 1회 기출문제 
13회 2016년 3회 기출문제
14회 2016년 2회 기출문제 
15회 2016년 1회 기출문제

핵심요약

1과목 컴퓨터 일반 
2과목 스프레드시트 일반
3과목 데이터베이스 일반 

※ 핵심요약은 PDF 파일로 제공됩니다. 핵심요약 PDF 사용을 위한 회원 가입 및 구입 도서 등록 절차는 9쪽을 참고하세요.
수험생의 마음으로 만든 책 - 시나공초단타 합격전략 - 시나공 기출문제집 시리즈- 시간이 부족한 수험생들의 궁금증 완전 해결!시나공 홈페이지(sinagong.gi

# 26 트렌드 코리아 2022 : 추천사 없음 [300]

In [35]:
url = 'http://www.yes24.com/Product/Goods/103737985'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/103737985?goodsSortNo=001013&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272922516'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



TIGER OR CAT  검은 호랑이처럼 힘차게 포효하는 2022가 되기를 “우리를 죽이지 못하는 것들은 우리를 더 강하게 만든다.”니체의 말이다. 거의 2년이 넘는 팬데믹 속에서도 우리는 삶을 이어왔고 고통과 절망 속에서도 미래에 대한 희망의 끈을 놓지 않았다. 획기적이고 거침없는 기술의 발전은 오히려 우리가 꿈꾸던 세상을 더욱 앞당기고 있다. 미증유의 전염병과 현명하게 공생하는 ‘위드 코로나’를 준비하면서 이제 우리는 더 강해지고 있다. 검은 호랑이의 해를 맞아, 서울대 소비트렌드분석센터는 10개의 키워드 두운을 “TIGER OR CAT”으로 잡았다. 팬데믹 위기 상황에 얼마나 잘 대처하느냐, 기업보다 진화의 속도가 더 빠른 소비자들의 니즈를 어떻게 맞출 것인가, 더 나아가 유권자들의 마음을 어떻게 얻을 것인가? 거침없이 포효하는 호랑이가 될 것인가, 고양이가 될 것인가? 우리 모두는 큰 갈림길에 놓여 있다.                     

교수, 트렌드 연구자, 컨설턴트, 작가. 서울대학교 생활과학대학 소비자학과에서 학생들을 가르치고, 생활과학연구소 소비트렌드분석센터를 이끌며 소비트렌드를 연구하고 있다. 서울대학교 교육상, 매일경제신문 정진기언론문화상, 한국소비자학회 최우수논문상, 한국정책학회 학술상, 한국갤럽 최우수논문지도상, 한국마케팅협회 공로상, 한중경영대상 한중경제협력상 등을 수상했다. 코웨이, 아모레퍼시픽, 에버랜드, 현대자동차, 삼성전자, SK텔레콤, LG전자, CJ그룹, 신한카드, 한국외식산업연구원, 신세계그룹, SK경영경제연구소, 롯데마트, 제일기획, 한라마이스터, AK플라자 등 여러 주요 기업들을 자문하며, 이론적 지식과 실무적 경험의 시너지를 도모하는 데도 힘을 쏟고 있다.KBS 해피FM에서 「김난도의 트렌드 플러스」라는 라디오 프로그램을 진행했으며, 한국형 대규모 온라인 공개강좌 K-MOOC에서 ‘소비자와 시장’이라는 강좌를 운영하고 있다. 최근에는 유튜브 채널 「트렌드코리아TV」를 개설해 트렌드 관련 콘텐츠를 업로드하고 있

# 27 세상의 마지막 기차역 : 추천사 없음 [800]

In [36]:
url = 'http://www.yes24.com/Product/Goods/109308932'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/109308932?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685272976159'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



2022년 전 서점 종합 베스트셀러 / 외국 소설 분야 1위
우리를 뜨겁게 울린 화제의 소설 ‘세상의 마지막 기차역’
20만 부 기념 리커버 에디션 전격 출간!

2022년 출간 후 단숨에 외국 소설 분야 1위, 종합 베스트셀러에 오르며 수많은 독자의 마음을 울린 『세상의 마지막 기차역』. “읽는 내내 눈물이 펑펑 쏟아졌다”, “주인공에 이입되어 그리운 사람이 떠올라 가슴이 먹먹해졌다”, “지금 내 곁에 있는 사랑하는 사람의 소중함을 일깨워주는 고마운 책” 등 독자들 사이에서 크게 입소문이 난 이 책은 2022년 20만 부 이상 팔렸으며, 현재도 그 인기를 이어나가고 있다. 

이번에 새롭게 출간된 『세상의 마지막 기차역』 리커버 에디션은 20만 부 판매 돌파를 기념한 것으로, 믿고 보는 일러스트레이터 반지수 작가와의 콜라보레이션을 통해 화사한 봄날을 배경으로 재탄생했다. 기차 사고가 일어날 줄을 꿈에도 모른 채 여느 때와 다름없는 평화로운 어느 날을 그려내고 있어 소설의 내용을 알게 되면 안타까움과 슬픔이 더 크게 다가온다. 동시에 사랑하는 사람을 한순간에 잃고 후회와 절망 속에서 잿빛 같은 하루하루를 살던 사람들이 ‘유령 기차’의 존재를 알게 되고 사고로 숨진 가족, 연인을 마지막으로 딱 한 번 다시 만날 수 있는 기회를 얻으면서 비로소 다시 살아갈 희망을 되찾는 과정이 표지의 분위기에 그대로 묻어나 보는 사람으로 하여금 가슴을 뭉클하게 한다. 

“평범한 아침 인사가 마지막이 될 줄 알았다면 그렇게 보내진 않았을 텐데….” 남겨진 사람과 떠나간 사람 모두가 ‘세상의 마지막 기차역’에서 만나 진정으로 슬픔을 치유하게 되는 가슴 따뜻한 휴먼 판타지 소설.

                                    

현실과 판타지를 자유자재로 넘나들며 몰입도 높은 이야기로 웃음과 감동, 슬픔과 재미를 선사하는 이야기 장인. 1978년 일본 효고현에서 태어나 간사이대학교 법학부를 졸업했다. 그 후 [폭소 레드카펫], [킹 오브 콩트], [좋은 아침입

# 28 인간 실격 : 추천사 없음 [800]

In [37]:
url = 'http://www.yes24.com/Product/Goods/1387488'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/1387488?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685273027609'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



인간의 나약함을 탁월하게 묘사하는 다자이 오사무의 작품을 새롭게 읽는다. 순수하고 여린 심성의 젊은이가 인간 사회의 위선과 잔혹성을 견디지 못하고 파멸되어 가는 과정을 그린 소설로, 어느 세계에도 속하지 못한 채 인간 실격자로 전락한 주인공의 내면을 치밀한 심리묘사로 기록하였다. 다자이 작품 속의 타락과 자기파괴적 언행은 제2차 세계대전에서 패망한 후 공황상태에 빠진 일본 젊은이들의 의식을 반영하고 있다. 

다자이 작품은 기성세대의 가치관 및 윤리관, 도덕관이 패전과 함께 붕괴되면서 기존 사회에 속한 모든 것을 거부하고 새로이 시작하고자 하는 처절한 몸부림을 담고 있다. 어떻게든 사회에 융화하고자 애쓰고, 인간에 대한 구애를 시도하던 주인공이 결국 모든 것에 배반당하고 인간 실격자가 되어가는 패배의 기록인 이 작품은 그런 뜻에서 현대 사회에 대한 예리한 고발 문학이라 할 수 있다.

함께 실린 작품, 「직소」는 ‘나약한 인간으로서의 유다’라는 새로운 시각을 보여준다. 유다가 예수를 고발하는 자리에서 늘어놓는 이야기를 마치 독자가 현장에서 함께 듣고 있는 것처럼 서술한 작품으로, 예수를 흠모하고 사랑했지만 그 사랑이 거부당한 데 대한 분노와 반발심으로 예수를 팔아넘기게 되는 유다의 갈등과 번민을 생생하게 묘사하고 있다.


                                    

1909년 6월 19일, 일본 아오모리 현 쓰가루 군 카나기무라에서 태어났다. 본명은 쓰시마 슈지[津島修治]이다. 그는 경제적으로 풍요로운 환경에서 성장했으나 가진 자로서의 죄책감을 느꼈고, 부모님의 사랑을 제대로 받지 못해서 심리적으로 불안정하게 성장한다. 1930년, 프랑스 문학에 관심이 있었던 그는 도쿄제국대학 불문과에 입학하지만, 중퇴하고 소설가가 되기로 결심한다. 이후 소설가 이부세 마스지[井伏_二]의 문하생으로 들어간 그는 본명 대신 다자이 오사무[太宰治]라는 필명을 쓰기 시작한다. 그는 1935년 소설 「역행(逆行)」을 발표하면서 본격적으로 작가의 길을 걷게

# 29 오늘 밤, 세계에서 이 사랑이 사라진다 해도 : 추천사 없음 [800]

In [38]:
url = 'http://www.yes24.com/Product/Goods/102360203'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/102360203?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685273084488'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



매일 기억을 잃는 너와
두 번 다시 돌아갈 수 없는 사랑을 했다
제26회 전격소설대상 ‘미디어워크스문고상’ 수상작

밤에 자고 일어나면 기억이 리셋되는 ‘선행성 기억상실증’을 앓는 소녀 히노 마오리와 무미건조한 인생을 살고 있는 평범한 고등학생 가미야 도루의 풋풋하고 애틋한 사랑 이야기. 선행성 기억상실증이라는 익숙한 소재를 매우 수준 높은 청춘 소설로 탄생시켰다는 극찬을 받으며 제26회 전격소설대상 ‘미디어워크스문고상’을 수상한 이 작품은 간질간질한 청춘의 로맨스를 전혀 예측할 수 없는 국면으로 끌고 가, 깐깐하고 엄격한 심사위원 모두를 눈물 흘리게 만들었다는 흥미로운 뒷이야기를 남긴 소설이기도 하다.

“날 모르겠지만, 사귀어줄래…?” 어쩔 수 없이 건넨 도루의 거짓 고백을 “날 정말로 좋아하지 말 것. 지킬 수 있어?”라는 조건을 걸고 허락한 히노. 조건부 연애였던 두 사람의 관계는, 연인이지만 연인이 아닌 이 특수한 관계는 ‘매일 기억이 사라지는’ 잔혹한 현실을 극복해나갈 수 있을까? 이들 사랑의 끝에 어떤 결과가 기다리고 있을까?




                                    

『오늘 밤, 세계에서 이 사랑이 사라진다 해도』로 제26회 전격소설대상 ‘미디어워크스문고상’을 수상하며 화려하게 데뷔했다. 매일 기억이 리셋되는 ‘선행성 기억상실증’이라는 자칫 진부할 수 있는 아이디어를 치밀한 구성과 전혀 예측할 수 없는 반전으로 풀어내 수준 높은 청춘 소설을 창조했다는 극찬을 받으며 뛰어난 신인 작가의 등장을 알렸다. 평범하지만 한없이 다정한 소년 가미야 도루가 사고로 기억장애를 앓는 소녀 히노 마오리를 만나 서툴지만 따뜻하게 마음을 쌓아가는 이 이야기는 봄의 벚꽃, 초여름의 자전거, 한여름의 불꽃놀이 등 청춘을 상징하는 풋풋한 풍경과 어우러져 독자에게 기분 좋은 설렘을 선사한다. 하지만 두 주인공의 모습이 사랑스러우면 사랑스러울수록 독자가 느낄 슬픔과 여운은 점점 더 깊어지는 아이러니 또한 경험하게 될 것이다. 『네가 마지막으

# 30 쉽게 배우는 알고리즘 : 출판사리뷰, 추천사 없음 [600]

In [39]:
url = 'http://www.yes24.com/Product/Goods/58154784'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/58154784?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685273130163'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



귀납적 사고를 통한 문제 해결 기법 훈련알고리즘에 대한 지식을 기반으로 제대로 프로그래밍을 하는 이들뿐만 아니라, 알고리즘 속에 깃들어 있는 여러 가지 생각하는 방법, 자료구조, 테크닉을 통해 체계적으로 생각하는 훈련을 하고자 하는 모든 이들을 대상으로 한다. 알고리즘의 설계와 분석을 활용하여 체계적으로 사고할 수 있는 빌딩 블록을 구축하여 컴퓨터 또는 관련 분야의 연구자 또는 개발자로서 갖춰야 할 지적 기반을 쌓을 수 있다.                    

서울대 컴퓨터공학부 교수이자 ㈜옵투스투자자문 대표. 컴퓨터 알고리즘 최적화 전공자인 문병로 교수는 자신의 분야에서 여러 세계 기록들을 갈아치운 권위 있는 전문가다. 다양한 응용연구를 진행하던 문병로 교수는 2000년 최적화 이론의 가장 복잡한 응용 예로 주식시장 데이터에 본격적으로 관심을 갖게 되었고 실물 주식시장에서 놀라운 운용 수익률을 보이고 있다. 기존의 금융 전문가와는 전혀 다른 배경으로 주식시장에 진입한 그는‘한국판 제임스 사이먼스’로 불린다. 제임스 사이먼스는 대표적인 글로벌 헤지펀드 회사인 르네상스 테크놀로지스의 설립자이자 회장이다. 금융 전공자가 아닌 사이먼스는 MIT 수학 박사이며 뉴욕주립대에서 수학과 학과장을 지냈고 베블렌상을 받은 수학 천재다. 그의 메달리온펀드는 워런 버핏을 압도하는 수익률을 올려 왔다. 문병로 교수를 제임스 사이먼스에 견주는 것은 그가 학문적 공학기술을 바탕으로 한 국내 최초의 알고리즘 트레이딩 시스템을 개발,실제로 놀라운 성과를 올리고 있기 때문이다.루머와 잡음투성이인 시장에서 투자자가 이익을 내기 위해서는, 수치와 확률에 근거한 영리한 접근이 필요하다. 하지만 공간의 방대함으로 인해, 이론 무장이 되지 않은 컴퓨터 프로그램이나 개인이 원하는 솔루션을 찾아내는 건 불가능하다. 문병로 교수는 ‘메트릭 스튜디오’라 부르는 그의 연구 공간에서 최첨단 기법을 이용하여 가장 매력적인 포트폴리오와 투자전략을 도출하고 있다.금융시장은 사람 주도형 투자에서 컴퓨터 주도형

# 31 쇼코의 미소 : 추천사 없음 [800]

In [40]:
url = 'http://www.yes24.com/Product/Goods/29288225'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/29288225?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276041343'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



“소설가로서 최은영의 가장 큰 미덕은 
그게 무슨 탐구든 반드시 근사한 이야기로 들려준다는 점이다. 
그녀가 앞으로 쓰게 될 근사한 이야기들이 바로 이 책에서 시작했다.” 
_김연수(소설가)

2013년 겨울, 『작가세계』 신인상에 중편소설 「쇼코의 미소」가 당선되어 등단, 그 작품으로 다음해 젊은작가상을 수상하면서 많은 사람들에게 특별한 인상으로 다가갔던 바로 그 신인 소설가 말이다. 그러나 이 ‘특별한 인상’은, 발표한 작품이라고는 등단작 「쇼코의 미소」 한 편밖에 없는 신인 작가가, 등단한 지 채 두 달이 되지 않은 시점에 젊은작가상을 수상했다는 것만을 의미하지 않는다. 저마다의 날카로운 감식안을 지닌 소설가와 평론가들로부터 공통의 감상을 이끌어냈다는 점에 그 특별함이 있다. 어떤 갑론을박도 없이 모두에게서 동일한 평가를 받는 작품이 탁월한 소설이라 말하려는 것이 아니다. 등단작에 대해 흔히 우리가 걸게 되는 기대―기존 작품과 구별되는 ‘낯섦’과 ‘전위’에 대한 요구―로부터 물러나, 별다른 기교 없이 담백하게 이야기를 풀어나가고, 그 정통적인 방식을 통해 읽는 이의 마음을 움직였다는 것에 「쇼코의 미소」가 지닌 특별함이 담겨 있다. 그러니까, “고레에다 히로카즈나 이누도 잇신 감독의 어떤 영화들처럼 거의 모든 영역에서 ‘진실하다’라는 느낌”(문학평론가 신형철)을 준다는 것, 그로부터 “소설이 주는 감동이란 무엇인가를 새삼 생각해보게 만들었다”(소설가 임철우)라는 것. 

최은영은 등단 초기부터, “선천적으로 눈이나 위가 약한 사람이 있듯이 마음이 특별히 약해서 쉽게 부서지는 사람도 있는 법”이라고, 전혀 짐작할 수 없는 타인의 고통 앞에 겸손히 귀를 열고 싶다고 밝혀왔다. 최은영의 시선이 가닿는 곳 어디에나 사람이 자리해 있는 것은 바로 이 때문일 터. 총 7편의 작품이 수록된 최은영의 첫 소설집 『쇼코의 미소』는 사람의 마음이 흘러갈 수 있는 정밀한 물매를 만들어냄으로써, 우리들을 바로 그 ‘사람의 자리’로 이끈다. 

                

# 32 부자 아빠 가난한 아빠 : 추천사 없음 [300]

In [41]:
url = 'http://www.yes24.com/Product/Goods/58774995'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/58774995?goodsSortNo=001013&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276110940'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



전 세계에서 가장 많이 팔린 경제경영서‘부자 아빠 가난한 아빠’ 20주년 기념 업그레이드 에디션 출간!- 세계 금융 변화에 맞춘 41개의 NEW 코멘트- 토론과 실천을 위한 10가지 스터디세션 등 500매 분량의 내용 추가1997년 미국에서 처음 출간된 이래 전 세계 51가지 언어로 109개국에서 출판되어 4000만 부(국내 350만 부)가 판매된 경제경영 재테크 분야 최고의 밀리언셀러 ‘부자 아빠 가난한 아빠’ 시리즈. 20년의 세월을 통해 검증된 부자들의 돈과 투자에 대한 지침들과 오늘날의 시대상에 맞춘 새로운 정보들이 추가된 『부자 아빠 가난한 아빠 20주년 특별 기념판』이 ㈜민음인에서 출간되었다. 지난 20년간 독자들에게 가장 많은 사랑을 받아 온 『부자 아빠 가난한 아빠 1』의 내용에 세계 금융 변화에 맞춘 41개의 ‘20년 전 그리고 오늘’과 토론과 실천을 위한 10가지 ‘스터디세션’ 등 원고지 500매 분량이 추가된 업그레이드 에디션이다. 돈에 대한 선입관을 깨뜨리는 파격적인 내용과 세월이 흘러도 변치 않는 투자의 원칙들은, 비트코인 광풍, 부동산 경매 시장의 활기 등 다양한 양상의 투자에 대한 관심이 높아지고 있는 오늘날에도 여전히 ‘돈’과 ‘투자’의 흔들리지 않는 기준이 되어 줄 것이다.무엇보다 이 책이 독자들에게 파고든 이유는 돈에 대한 일반적인 상식을 뒤집었기 때문이다. 저자는 사람들이 가난한 이유는 돈이 없어서가 아니라, 돈을 금기시하며 금융 지식을 멀리하는 사고와 문화 때문이라고 말한다. 저자의 가난한 아버지는 공부를 많이 했지만, 늘 카드대금 청구서와 주택융자금에 시달렸다.(“돈을 좋아하는 것은 모든 악의 근원이다. 공부 열심히 해서 좋은 직장을 구해야지. 위험을 피하고 안정적으로 살아라.”) 반면 친구의 부자 아버지는 정규 교육은 제대로 받지 못했지만 금융 IQ를 터득하여 막대한 부를 쌓았다.(“돈이 부족한 것은 모든 악의 근원이다. 공부 열심히 해서 좋은 회사를 차려라. 네가 똑똑한 사람을 고용해야 한다. 무엇보다 위험을 관리하

# 33 (이기적) 컴퓨터활용능력 : 추천사 없음 [600]

In [42]:
url = 'http://www.yes24.com/Product/Goods/103649099'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/103649099?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276184436'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



현 출제 기준을 완벽하게 반영한 ‘2022 이기적 컴퓨터활용능력 1급 필기 최신 문제집’이 출간되었다. 본 도서는 컴퓨터활용능력 시험을 가장 효율적으로 준비할 수 있도록 구성된 도서다. 21년 상시 시험을 분석하여 구성한 ‘기출 변형 문제 8회분’과 ‘최신 기출문제 15회분’을 통해 실전 감각을 익힐 수 있도록 하였으며, ‘자주 출제되는 기출문제 120선’, ‘핵심이론 POINT 148선’, ‘실습을 통해 이해하는 엑셀 & 액세스 주요 기능’을 통해 이론 또한 빠짐없이 준비할 수 있도록 하였다. 특히 2022년판은 문제 풀이 동영상 강의를 대폭 추가하여 이론 동영상 강의와 함께 최적의 학습을 할 수 있도록 하였다. 이처럼 이기적 컴퓨터활용능력 필기 최신문제집 도서는 최적의 학습을 돕는 합격 전문서다.

                                    

2008년부터 이기적 컴퓨터활용능력 필기 도서를 담당하고 있다. 이패스코리아에서 컴퓨터활용능력 강의를 하고 있으며, 강원교육과학정보원 SME, 고용노동부 직훈 교사를 역임하고 있다. 2019 교육부장관 표창장을 수상했으며, 컴퓨터활용능력 필기, 정보처리기사 필기, e-Test Professionals MS-EXCEL 등을 집필하였다.
핵심 이론 POINT 148선(동영상 강의 제공)

1과목 컴퓨터 일반
2과목 스프레드시트 일반
3과목 데이터베이스 일반

실습을 통해 이해하는 엑셀&액세스 주요 기능

자주 출제되는 기출문제 120선(동영상 강의 제공)

기출 변형 문제 8회(동영상 강의 제공)

2021년 기출 변형 문제 1회
2021년 기출 변형 문제 2회
2021년 기출 변형 문제 3회
2021년 기출 변형 문제 4회
2021년 기출 변형 문제 5회
2021년 기출 변형 문제 6회
2021년 기출 변형 문제 7회
2021년 기출 변형 문제 8회

최신 기출문제 15회(동영상 강의 제공)

제1회 최신 기출문제(2020년 7월 5일 시행)
제2회 최신 기출문제(2020년 2월 29일 시행)

# 34 핸즈온 머신러닝 [600]

In [43]:
url = 'http://www.yes24.com/Product/Goods/89959711'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/89959711?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276229595'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



머신러닝 전문가로 이끄는 최고의 실전 지침서 텐서플로 2.0을 반영한 풀컬러 개정판 『핸즈온 머신러닝』은 지능형 시스템을 구축하려면 반드시 알아야 할 머신러닝, 딥러닝 분야 핵심 개념과 이론을 이해하기 쉽게 설명한다. 사이킷런, 케라스, 텐서플로를 이용해 실전에서 바로 활용 가능한 예제로 모델을 훈련하고 신경망을 구축하는 방법을 상세하게 안내한다. 장마다 제공하는 연습문제를 풀며 익힌 내용을 확인하고 응용할 수도 있다. 머신러닝을 배우고 싶지만 어디서부터 시작해야 할지 막막하다면, 이 책이 인공지능 마스터로 가는 길에 좋은 친구가 될 것이다. 이번에 출간된 2판은 텐서플로 2를 반영하고 비지도 학습, 자연어 처리, 생성적 적대 신경망(GAN) 등 최신 기법을 추가했다. 사이킷런과 텐서플로 2에 더해 케라스를 사용하며, 예제 코드도 소프트웨어 최신 버전에 맞춰 갱신했다. 1부(머신러닝)에는 1개 장이 추가되었고, 2부(신경망과 딥러닝)는 최신 딥러닝 기법을 방대하게 수록하여 대폭 개정되었다. 부록 2개 장이 추가되었으며, 시각적 편의를 위해 전면 컬러로 인쇄했다.                    

머신러닝 컨설턴트. 2013년에서 2016년까지 구글에서 유튜브 동영상 분류팀을 이끌었다. 2002년에서 2012년까지 프랑스의 모바일 ISP 선두 주자인 Wifirst를 설립하고 CTO로 일했다. 2001년에는 Polyconseil을 설립하고 CTO로 일했다. 이 회사는 현재 전기차 공유 서비스인 Autolib’을 운영한다. 그 전에는 재무(JP 모건과 소시에테 제네랄), 방위(캐나다 DOD), 의료(수혈) 등 다양한 분야에서 엔지니어로 일했다. C++, WiFi, 인터넷 구조에 대한 몇 권의 기술 서적을 썼으며 한 프랑스 공과대학교에서 컴퓨터 과학을 가르쳤다.
[PART 1 머신러닝]

CHAPTER 1 한눈에 보는 머신러닝
1.1 머신러닝이란? 
1.2 왜 머신러닝을 사용하는가?
1.3 애플리케이션 사례
1.4 머신러닝 시스템의 종류
1.5 머신러닝의 주요

# 35 (2022 에듀윌) 한국사능력검정시험 all기출 : 작가소개, 추천사 없음 [900]

In [44]:
url = 'http://www.yes24.com/Product/Goods/105517511'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/105517511?goodsSortNo=001001&resourceKeyGb=01&goodsStateGb=05&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276281725'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



기출은 충실하게, 합격은 빠르게!에듀윌이 만든 시험에 강한 프리미엄 기출문제집!모든 시험 대비의 기본은 그 시험에 이미 출제된 문제를 학습하는 것이다. 기출문제를 통해 출제자들이 어떤 개념을 중요하게 생각하는지, 다음 시험에는 어떤 문제가 출제될지를 예상해 볼 수 있기에 기출문제의 중요성은 변하지 않다. 시험이 임박했을 때, 가장 많은 수험생들의 선택을 받은 것은 역시나 에듀윌 한국사능력검정시험 기출문제집이었다. 에듀윌은 1위의 자리에 만족하지 않고, 수험생들을 위해 끊임없이 고민하였다. 그 결과, 수험생들이 한 번에 합격할 수 있도록 기존보다 더욱 업그레이드된 기출문제집을 완성하였다. 또한 수험생들이 장소와 시간에 구애받지 않고 학습에 전념할 수 있도록 ‘회차별 해설강의’ QR코드 제공 서비스를 전격 공개한다. 어디에도 없는 ‘회차별 해설강의 바로접속 서비스’와 프리미엄 기출풀이로 한 번에 합격 가능한 ‘ALL기출’의 효과를 지금 바로 확인하자!                    

없음
[회차별 해설강의(QR코드)]

55~31회

[기출문제]

제55회 기출문제
제54회 기출문제
제52회 기출문제
제51회 기출문제
제50회 기출문제
제49회 기출문제
제48회 기출문제
제47회 기출문제

[첨삭해설]

제55회 기출문제
제54회 기출문제
제52회 첨삭해설
제51회 첨삭해설
제50회 첨삭해설
제49회 첨삭해설
제48회 첨삭해설 
제47회 첨삭해설

[특별제공(PDF)]
 
부가학습자료1 아이콘연표
부가학습자료2 핵심이론 요약집
기존의 한국사능력검정시험 교재에서 볼 수 없던 새로운 구성의 기출문제집! [기출문제]+[첨삭해설]+[단기합격팩]의 프리미엄 구성으로 한능검 합격 완벽 대비!- [회차별 해설강의 QR코드] 31~55회까지 회차별 해설강의 바로 학습 가능31회부터 55회까지의 무료 해설강의 QR코드를 한 곳에 모아 제공합니다. QR코드만 찍으면 총 24회 분량의 고퀄리티 무료 해설강의를 모바일로 편하고 빠르게 수강할 수 있습니다. - [기출문제] 합격

# 36 이것이 취업을 위한 코딩 테스트다 with 파이썬 [600]

In [45]:
url = 'http://www.yes24.com/Product/Goods/91433923'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/91433923?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276330568'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



나동빈 저자의 유튜브 라이브 방송 https://www.youtube.com/c/dongbinna

IT 취준생이라면 누구나 입사하고 싶은 카카오 · 삼성전자 · 네이버 · 라인!
취업의 성공 열쇠는 알고리즘 인터뷰에 있다!
	
IT 취준생이라면 누구나 가고 싶어 하는 카카오, 라인, 삼성전자의 2016년부터 2020년까지의 코딩 테스트와 알고리즘 대회의 기출문제를 엄선하여 수록했다. 최근 5년간의 코딩 테스트 기출문제를 분석하여 반드시 알아야 하는 알고리즘을 8가지로 정리했다. 8가지 핵심 알고리즘 이론을 쉽게 설명하고, 관련 실전 문제를 풀이했다. 출제 유형 분석, 이론 설명, 기출문제 풀이까지! 어떤 코딩 테스트도 대비할 수 있을 것이다. 코딩 테스트에서 주로 선택하는 파이썬을 기반으로 설명되어 있으며, 파이썬 코드 외에도 C/C++, 자바 코드를 추가로 제공한다.
                                    

욕심 많은 그는 개발자, 유튜버, 강사 그리고 대학원생까지 1인 4역을 소화하고 있다. 한국인터넷진흥원(KISA), 프로그래머스, 패스트캠퍼스, 삼성멀티캠퍼스, KG에듀원, 국내 소프트웨어 마이스터고등학교 등 다양한 현장에서 코딩 테스트를 비롯해 IT 관련 주제를 강의했다. 학부 시절에는 각종 IT 대회에 출전해 경험을 쌓았고, 졸업 후 개발자로 직장 생활을 하다가 공부에 대한 부족함과 욕구가 생겨 현재 포항공과대학에서 석사 과정 중이다. 2015년부터 유튜브에서 개발 채널을 운영해왔고, 어느덧 9만 명이 넘는 구독자가 참여하고 있다.
__지은이의 글
__리뷰어의 글
__들어가기 전에
__이 책의 구성
__동영상 강의 및 Q&A


PART 01 코딩 테스트, 무엇을 어떻게 준비할까?

Chapter 01 코딩 테스트 개요
__1 코딩 테스트 개념과 배경
__2 실습 환경 구축하기
__3 복잡도

Chapter 02 16~20년 코딩 테스트 기출문제 유형 분석
__1 최신 출제 경향과 준비 방향
__2 연도별 코딩 테스트 

# 37 정의란 무엇인가 : 추천사 없음 [300]

In [46]:
url = 'http://www.yes24.com/Product/Goods/15156691'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/15156691?goodsSortNo=001009&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276376731'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



한국에 ‘정의’ 열풍을 불러일으킨 마이클 샌델은 구제 금융, 대리 출산, 동성 결혼, 과거사 공개 사과 등 현대 사회에서 우리가 흔히 부딪히는 문제를 통해 ‘무엇이 정의로운가’에 대한 해답을 탐구했다. 이 책은 탁월한 정치 철학자들이 남긴 시대를 초월한 철학적인 질문을 알기 쉽게 소개한다. 이를 통해 옳고 그름, 정의와 부당함, 평등과 불평등, 개인의 권리와 공동선을 둘러싼 주장들이 경쟁하는 공적 담론과 토론의 장에서 정의에 관한 자신만의 견해를 정립하고 논리 기반을 굳건하게 다지는 토대를 제공한다. 이 책은 현대 사회의 문제를 진단하고 새로운 대안을 찾아내는 정치 철학자들의 지적 탐색 과정을 보여준다.                    

2010년 이후, 한국에 ‘정의’ 열풍을 불러일으켰다. 27세에 최연소 하버드대학교 교수가 되었고, 29세에 자유주의 이론의 대가인 존 롤스의 정의론을 비판한 『자유주의와 정의의 한계』(1982)를 발표하면서 세계적으로 명성을 얻었다. 1980년부터 하버드대학교에서 정치철학을 가르치고 있으며, 그의 수업은 현재까지 20여 년 동안 학생들 사이에서 최고의 명강의로 손꼽힌다. 존 롤스 이후 정의 분야의 세계적 학자로 인정받는 그는 명실공히 이 시대의 최고 석학이자 철학계의 록스타이다. 대표 저서로 『정의란 무엇인가』 『돈으로 살 수 없는 것들』 『정치와 도덕을 말하다』 『완벽에 대한 반론』 『정의의 한계』등이 있다.『마이클 샌델, 중국을 만나다』 는 중국 철학 연구자들이 마이클 샌델의 이론과 저작을 동양 철학의 시각으로 분석한 평론과 그에 대한 샌델의 답변을 함께 모은 것이다. 동서양의 철학적 대화를 살펴봄으로써 마이클 샌델의 ‘정의’를 새롭게 경험할 수 있을 것이다.
이 책에 쏟아진 찬사  /  한국어판 서문

1장 정의란 옳고 그름을 판단하는 문제일까?

2장 최대 행복 원칙: 공리주의

3장 우리는 우리 자신을 소유하는가?: 자유지상주의

4장 대리인 고용: 시장 논리의 도덕성 문제

5장 동기를 중시하는 시각: 이마

# 38 구의 증명 : 추천사, 회원리뷰 없음 [800]

In [47]:
url = 'http://www.yes24.com/Product/Goods/118578901'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

print("회원리뷰 없음")



만약 네가 먼저 죽는다면나는 너를 먹을 거야.그래야 너 없이도 죽지 않고 살 수 있어.사랑 후 남겨진 것들에 관한 숭고할 만큼 아름다운 이야기최진영 소설 『구의 증명』은 사랑하는 연인의 갑작스러운 죽음 이후 겪게 되는 상실과 애도의 과정을 통해 삶의 의미 혹은 죽음의 의미를 되묻는 소설이다. 이 작품에서 최진영은 퇴색하지 않는 사랑의 가치를 전면에 내세우고 아름다운 문장과 감성적이며 애절한 감수성을 통해 젊고 아름다운 남녀의 열정적인 사랑과 냉정한 죽음에 대해 세련된 감성과 탁월한 문체로 담아내고 있다.                    

1981년 눈이 많이 내리던 날 서울에서 태어났다. 낮엔 일하고 밤엔 글 쓰다가 2006년 [실천문학]으로 등단했다. 소설집 『팽이』, 『겨울방학』, 장편소설 『당신 옆을 스쳐간 그 소녀의 이름은』, 『끝나지 않는 노래』, 『나는 왜 죽지 않았는가』, 『구의 증명』, 『해가 지는 곳으로』, 『이제야 언니에게』, 『내가 되는 꿈』, 『팽이』, 『겨울방학』 등을 썼다. 앤솔러지 『장래 희망은 함박눈』을 함께 썼다. 박범신, 공지영, 황현산 등 심사위원들의 만장일치로 제15회 한겨레문학상에 당선되었으며, 만해문학상, 백신애문학상, 신동엽문학상 등을 수상했다.
구의 증명
작가의 말
상상을 가능케 하는 사랑그런 사랑을 가능케 하는 상상여자와 남자가 등장한다. 관형사 ‘한’이 아닌 대명사 ‘이’ 사람일 수밖에 없는 관계. 그런 사람들이 있다. 그걸 운명이라 말할 수도 있겠고, 대수롭지 않게 연인이라고 잘라 말할 수도 있겠으나 소설에서의 두 주인공 ‘구(남자)’와 ‘담(여자)’은 그 낱말의 범위에서 조금은 이탈해 보인다. 그들은 회문(回文)처럼 영원히 같이 붙어 원의 둘레를 순환할 수밖에 없는 관계. 타인이 만들어낸 우연과 엇갈림 등속을 겪지만 삶의 곡선 위에 놓인 두 개의 점은 궤도가 같기에 그들의 운명 또한 같을 수밖에 없어 어떻게든 만나게 된다. 누구는 그런 관계를 지독하다 할지 모르고 또 누구는 완전한 사랑이라 말할 수도 있

# 39 트렌드 코리아 2023 : 추천사 없음 [300]

In [48]:
url = 'http://www.yes24.com/Product/Goods/113416767'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/113416767?goodsSortNo=001013&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276463588'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



RABBIT JUMP 더 높은 도약을 준비하는 검은 토끼의 해     세계화의 종말, 갈등과 분열, 그리고 전쟁. 수십 년간 이어져 온 평화와 공존의 시대는 막을 내리고 엄청난 위기감 속에서 사람들은 다가올 미래를 두려워한다. 자산시장 및 증시의 버블붕괴는 마치 2008년 글로벌 경제위기의 데자뷔를 보는 듯하다. 제2의 외환위기 경고도 들려온다. 매우 부정적인 전망이 압도하는 2023년을 목전에 두고 있는 지금, 소비 트렌드 전망에서 가장 중요한 것은 무엇이 반복되고 무엇이 달라질 것인가를 구별하는 작업일 것이다. 즉, ‘불황기의 소비 패턴’을 과거와 비교해보는 것이고, 또 하나는 ‘소비의 전형성’이 사라지는 시대의 흐름을 분석하는 것이다. 이른바 ‘평균 실종’이 가장 첫 번째 키워드인 이유다. 변화의 속도가 더욱 빨라지는 2023 대한민국. 소비자들은 어떤 선택을 할 것인가?                     

교수, 트렌드 연구자, 컨설턴트, 작가. 서울대학교 생활과학대학 소비자학과에서 학생들을 가르치고, 생활과학연구소 소비트렌드분석센터를 이끌며 소비트렌드를 연구하고 있다. 서울대학교 교육상, 매일경제신문 정진기언론문화상, 한국소비자학회 최우수논문상, 한국정책학회 학술상, 한국갤럽 최우수논문지도상, 한국마케팅협회 공로상, 한중경영대상 한중경제협력상 등을 수상했다. 코웨이, 아모레퍼시픽, 에버랜드, 현대자동차, 삼성전자, SK텔레콤, LG전자, CJ그룹, 신한카드, 한국외식산업연구원, 신세계그룹, SK경영경제연구소, 롯데마트, 제일기획, 한라마이스터, AK플라자 등 여러 주요 기업들을 자문하며, 이론적 지식과 실무적 경험의 시너지를 도모하는 데도 힘을 쏟고 있다.KBS 해피FM에서 「김난도의 트렌드 플러스」라는 라디오 프로그램을 진행했으며, 한국형 대규모 온라인 공개강좌 K-MOOC에서 ‘소비자와 시장’이라는 강좌를 운영하고 있다. 최근에는 유튜브 채널 「트렌드코리아TV」를 개설해 트렌드 관련 콘텐츠를 업로드하고 있다.12년째 [트렌드 코리아] 시리즈

# 40 (Do it!) 점프 투 파이썬 : 추천사 없음 [600]

In [49]:
url = 'http://www.yes24.com/Product/Goods/74419916'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/74419916?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276596303'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



파이썬 4년 연속 베스트셀러 1위! 『Do it! 점프 투 파이썬』 전면 개정판 출시!문과생도 중고등학생도 직장인도 프로그래밍에 눈뜨게 만든 바로 그 책이 전면 개정판으로 새로 태어났다! 2016년 《Do it! 점프 투 파이썬》으로 출간되었던 이 책은 약 4년 동안의 피드백을 반영하여 초보자가 더 빠르게 입문하고, 더 깊이 있게 공부할 수 있도록 개정되었다. 특히 ‘나 혼자 코딩’과 ‘코딩 면허 시험 20제’ 등 독자의 학습 흐름에 맞게 문제를 보강한 점이 눈에 띈다. 실습량도 두 배로 늘었다.4년 동안 압도적 1위! 위키독스 누적 방문 200만! 수많은 대학 및 학원의 교재 채택 등! 검증은 이미 끝났다. 코딩을 처음 배우는 중고등학생부터 나만의 경쟁력이 필요한 문과생, 데이터 분석과 인공지능/머신러닝으로 커리어를 뻗어 나가고 싶은 직장인까지! 프로그래밍의 세계에 풍덩 빠져 보자.                    

국내 저자로는 최초로 2001년, 파이썬 안내서인 《점프 투 파이썬》을 출간하였다. 이후 직접 제작한 온라인 서비스 위키독스에서 온라인 독자와 소통하며 10년 동안 책을 다듬었고, 이 내용을 담아 2016년 《Do it! 점프 투 파이썬》을 출간하였다. 《Do it! 점프 투 파이썬》은 출간 후 지금까지 파이썬 분야는 물론 전체 프로그래밍 분야에서 1위를 꾸준히 유지하며 많은 독자를 프로그래밍의 세계에 입문시켰다.이번 전면 개정에서는 출간 후 약 4년 동안의 피드백을 반영하여 초보자가 더 빠르게 입문하고, 더 깊이 있게 공부할 수 있도록 책을 재구성하였다.프로그래밍 및 IT 관련 지식을 공유하는 위키독스(https://wikidocs.net)와 프로그래밍 문제풀이를 통해 코딩 실력과 알고리즘을 수련하는 코딩 도장(http://codingdojang.com) 서비스도 운영하고 있다. 현재는 뱅킹 프로그램을 만들며 프로그래머의 삶을 살고 있다.
첫째마당 파이썬 기본 문법 익히기

01장 파이썬이란 무엇인가?
__01-1 파이썬이란?
__01-

# 41 Visual C#.NET 프로젝트 실무 연습 : 저자소개, 출판사 리뷰, 추천사 없음 [600]

In [50]:
url = 'http://www.yes24.com/Product/Goods/78568960'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/78568960?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=04&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276695578'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



이 책은 초보자에게는 조금 벅찬 프로젝트들을 따라하도록 하고 있다. 이렇게 프로젝트를 따라하다 보면 자연스럽게 문법과 프로그래밍 언어의 응용 및 실무에 관한 전반적인 내용을 이해할 수 있기 때문이다．일반 바이블형 서적이 'C#.NET의 전체적 설명하기' 가 목적이라면 이 책의 목적은 한마디로 'C#.NET를 이용한 응용 프로그래밍 작성과 실무 연습'이 되겠다.                    

없음
Chapter 1   Visual C#.NET 살펴보기
　　　　　Project 1 폼 생성기 

Chapter 2   C#과 .NET 컨트롤
　　　　　Project 1 그림 보기
　　　　　Project 2 셈셈게임

Chapter 3   C#과 메시지
　　　　　Project 1 하노이탑 게임

Chapter 4   C#과 파일
　　　　　Project 1 문서 편집기
　　　　　Project 2 영문타자 연습기

Chapter 5   C#과 Windows API
　　　　　Project 1 프로그램 관리기
　　　　　Project 2 미니 탐색기

Chapter 6   C#과 그래픽
　　　　　Project 1 드로잉 툴(Drawing Tool)
　　　　　Project 2 이미지 유틸리티
　　　　　Project 3 사다리타기
　　　　　Project 4 그림맞추기 게임

Chapter 7   C#과 데이터베이스
　　　　　Project 1 팁 관리기
　　　　　Project 2 명함관리 프로그램

Chapter 8   C#과 .NET 컨트롤  제작
　　　　　Project 1 이미지클립 컨트롤
　　　　　Project 2 확장 텍스트박스
없음
없음
일단 이 책은 이전에 나왔던 좀더 비싸고 두꺼운 책을 줄인 것입니다. C#  기초없이 본다면 폭망입니다.  기초가 있어도  다른 유능한 사람에게 배우면 다행이지만 혼자하기엔 오타도 있기 때문에 공부하기 어렵습니다. 예를들어 1강부터 폼디자인을 하는데 도구상자에서 어떻게 끌어서 폼을 배치하는가에 대한 설명이 없습니다. 저야 이전에  

# 42 (큰별쌤 최태성의 별★별한국사) 한국사능력검정시험 : 추천사 없음 [900]

In [51]:
url = 'http://www.yes24.com/Product/Goods/116206629'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/116206629?goodsSortNo=001001&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276786078'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



『2023 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화 상』오랜 연구와 검증으로 한국사에 기초가 전혀 없는 사람도 단기간에 한능검에 합격할 수 있는 최적의 구성을 완성하였다. 판서의 장인 큰별쌤 최태성이 핵심만을 모아 만든 아트 판서를 수록하고, 시험에 자주 등장하는 사료, 사진, 지도 등을 모아 해설하여 합격을 위한 필수 개념을 익히기 쉽게 하였다. 또한, 한능검 기출문제를 모두 분석한 후, 기출 선택지를 이용한 별 채우기 문제를 수록하여 자연스럽게 기출 선택지가 기억될 수 있도록 하였고, 반복적으로 출제되는 주제의 최신 기출문제를 꼼꼼하고 친절한 해설과 함께 담아 실전에 대비할 수 있도록 하였다. 여기에 한국사 연표와 지역의 역사를 담은 지도를 통해 한국사 흐름잡기가 더욱 쉬울 것이다.제시된 이미지는 책의 뒷표지이다. 많은 사람들이 쓰는 책의 앞표지에 선한 메시지가 담기면 좋겠다는 큰별쌤의 뜻에 따라 『2023 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상』의 앞표지는 한국해비타트의 '주거환경개선 사업'에 대한 이야기로 구성되었다.                    

성균관대학교 사학과를 졸업하고, 고등학교 교실에서 20년간 학생들과 호흡하다 2001년부터 EBS 한국사 강의를 시작했다. 2017년에는 누구나 쉽고 편하게 역사 강의를 들을 수 있어야 한다는 생각으로 무료 온라인 강의 사이트 ‘모두의 별★별 한국사’와 무료 유튜브 강의 채널 ‘별별 히스토리’를 열었다. 보는 이를 유쾌하게 만드는 특유의 에너지와 균형 잡힌 역사관, 강의마다 뭉클한 감동을 선사하는 탁월한 스토리텔링 능력까지. 그는 고단한 삶에 지친 600만 수험생에게 ‘실질적 도움’과 ‘따듯한 응원’을 전해온 교사인 동시에, 대중 역사서 집필을 통해 ‘역사의 대중화’를 실천해온 작가이기도 하다. 일반 독자를 위한 역사서인 전작 《역사의 쓸모》는 23만 부 이상의 판매를 올리며 역사가 우리에게 살아 있는 쓸모임을 입증했다. 2008년과 2012년 두 차례에 걸쳐 

# 43 로보틱스 입문 : 저자소개, 출판사 리뷰, 추천사, 회원 리뷰 없음 [600]

In [52]:
url = 'http://www.yes24.com/Product/Goods/97798093'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")
print("회원리뷰없음")



이 책은 로봇 공학을 가르치는 교육 현장에서 가장 널리 사용되는 교재로서 기계적 머니퓰레이션 분야의 과학과 공학을 소개한다. 1장은 로보틱스 분야의 입문에 관한 내용이고, 2장에서는 3차원 공간 내에서의 위치와 방위를 표시하기 위해 사용되는 수학을 다룬다. 3장과 4장에서는 기계적 머니퓰레이터의 기하학을 다루고, 5장에서는 기구학을 속도와 정력에 대해 확장한다. 6장에서는 머니퓰레이터가 운동을 하는 데 필요한 힘과 모멘트를 다루며, 7장에서는 머니퓰레이터의 운동을 공간상의 궤도를 사용하여 표시하는 데 초점을 둔다. 8장에서는 머니퓰레이터의 기계적인 설계와 관련된 주제를 다루고, 9장과 10장에서는 공간에서 원하는 위치 궤도를 잘 따라갈 수 있도록 머니퓰레이터를 제어하는 방법을 살펴본다. 11장에서는 머니퓰레이터의 능동적 힘 제어를 다루고, 12장에서는 로봇을 프로그래밍하는 방법을 개관하며, 13장에서는 인간과 로봇 인터페이스의 최신 발전 분야인 오프라인 시뮬레이션과 프로그램 시스템을 소개한다.                    

없음
1. 도입
2. 공간 표시와 변환
3. 머니퓰레이터의 기구학
4. 머니퓰레이터의 역기구학
5. 야코비안: 속도와 정력
6. 머니퓰레이터 동역학
7. 궤도 생성
8. 머니퓰레이터 기구 설계
9. 머니퓰레이터의 선형 제어
10. 머니퓰레이터의 비선형 제어
11. 머니퓰레이터의 힘 제어
12. 로봇 프로그래밍 언어와 시스템
13. 오프라인 프로그래밍 시스템
없음
없음
회원리뷰없음


# 44 생산운영관리 프로세스와 공급사슬 : 책소개, 저자소개, 출판사 리뷰, 추천사 없음 [300]

In [53]:
url = 'http://www.yes24.com/Product/Goods/89494037'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/89494037?goodsSortNo=001004&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276881522'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))

없음
없음
1 가치 창출 행위로서의 운영 
 보론 A 의사결정 

PART 1 프로세스 관리 
2 프로세스 전략과 분석 
3 품질경영 
4 용량계획 
5 제약관리 
6 린 시스템 
7 프로젝트 관리 

PART 2 고객수요 관리 
8 예측 
9 재고관리 
 보론 B 특수재고모형 
10 운영계획과 일정계획 
11 자원계획 

PART 3 공급사슬 관리 
12 공급사슬 설계 
13 공급사슬과 입지 선정 
14 공급사슬 통합 
15 지속 가능한 공급사슬 관리
없음
없음
석정 출판사의Lee J. Krajewski 등저/백승규 등역의생산운영관리 공급사슬 및 프로세스 리뷰입니다.

가독성이 좋아 책에서 필요한 정보를 찾기가 편했습니다. 예제가 많아 직접 문제를 풀어보고 해설을 볼 수 있어서 공부하기 좋았습니다.

배송이 많이 밀려 예상하던 날짜에 받을 수 없어 약간 불안했습니다. 필요하신 분은 미리미리 주문하는게 좋을 것 같습니다.더보기
석정 출판사의 Lee J. Krajewski 등저/백승규 등역의 생산운영관리 공급사슬 및 프로세스 리뷰입니다.가독성이 좋아 책에서 필요한 정보를 찾기가 편했습니다. 예제가 많아 직접 문제를 풀어보고 해설을 볼 수 있어서 공부하기 좋았습니다.배송이 많이 밀려 예상하던 날짜에 받을 수 없어 약간 불안했습니다. 필요하신 분은 미리미리 주문하는게 좋을 것 같습니다.


# 45 오픽노잼 : 추천사 없음 [400]

In [54]:
url = 'http://www.yes24.com/Product/Goods/97247006'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/97247006?goodsSortNo=001011&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276958823'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



수백만 개의 오픽 질문을 단 4개로 정리했으며 실제 원어민이 쓰는 꿀표현과 꿀팁이 담겨 있다. 실제 원어민과 학생들의 답변을 바탕으로 설명했고 글로 한 번, 영상으로 두 번 공부하는 책이다. 스크립트를 외우지 않고도 오픽 답변하는 방법으로 설명했다.                     

前 아리랑 Korea TV (Canada, Toronto)前 삼육 SDA 어학원 강사前 삼성전자 오픽 인텐시브 전문강사前 유튜브 크리에이터(채널: 오픽노잼)前 오픽노잼스쿨 대표저서오픽노잼(스크립트 없이 오픽 쌉가능)
Preface
책 사용설명서 (책의 장점)

[CHAPTER 01 필러]
01. IM 시리즈 13 : 오픽답변에서 스크립트 냄새가 나면 응시료 버리는 거다
     - 스크립트, 과연 독일까 득일까?
02. 외국인편 1 : 이거 하나로 오픽 IH/AL 겁나 쉽게 받기!
     - IH에서 AL로 가는 지름길, 필러
03. 외국인편 5 : 필러 사용 방법, 음악문제로 연습하자!
     - 필러는 샷 추가다!
04. IM 시리즈 16 : IM2 받았다고 기뻐하지 마세요
     - 필러를 쓴 답변과 쓰지 않은 답변의 차이
05. 알아두면 좋은 필러

[CHAPTER 02 메인 포인트]
01. IM 시리즈 17 : 오픽에서 가장 중요한 영어 공부 방법
     - 메인 포인트(MP)가 대체 뭐길래?
     - 메인 포인트는 어떻게 공부해야 할까?
     - 메인 포인트를 빛나게 할 수 있는 전략? 초초반 전략!
02. IM 시리즈 14 : 오픽 시험 전에 꼭 봐야 할 총정리 영상
     - 모든 질문에 메인 포인트를 똑같이 말해야 할까?
     - 현재와 과거를 비교하는 질문의 메인 포인트를 알아보자
03. IM 시리즈 3 : 오픽 IM/AL를 벗어나는 룰
     - 솔직히… 잘 기억나지 않아요 VS 메인 포인트, 어느 것을 더 먼저 말해야 할까?
04. IM 시리즈 19 : 이 영상 보면 오픽 공부 방법을 바꾸게 됨
     - 그래서 메인 포인트는 

# 46 C# 교과서 : 추천사 없음 [600]

In [55]:
url = 'http://www.yes24.com/Product/Goods/90339705'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/90339705?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685276998792'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



기초부터 활용까지, C# 기본기를 탄탄하게 다진다!

20년 경력 전문가가 선별한 「핵심 내용」과 「학습 순서」로 구성!

Visual Studio 2019 + C# 8.0, 최신 버전으로 배우자.

『C# 교과서』는 생애 첫 프로그래밍 언어로 C#을 시작하는 독자를 대상으로 한다. 특히 응용 프로그래머를 위한 C# 입문서로, C#을 사용하여 게임(유니티), 웹, 모바일, IoT 등을 개발할 때 필요한 C# 기초 문법을 익히고 기본기를 탄탄하게 다지는 것이 목적이다. Microsoft MVP이자 20년 경력의 개발/강의 전문가인 저자가 선별한 ‘핵심 내용’과 ‘학습 순서’로 설명한다. 또한, 600개가 넘는 코드 조각과 실습 예제를 직접 실행하며 실력을 향상시킬 수 있다. C#을 접한 적이 있는 독자라도 알고 있는 내용을 한 번 더 정리할 기회가 될 것이다. 출간 이후 저자 블로그(dotnetkorea.com)에서 제공하는 실시간 온라인 강의(유튜브로 진행, 저자 블로그 공지) 등도 놓치지 말자.
유튜브 동영상 강의 구독신청 URL http://bit.ly/csharp-master
해당 유튜브에 C#교과서의 본문 전체 강의가 있습니다.                                    

Microsoft MVP. Visual Studio 분야 Microsoft 공인 강사(MCT)를 거쳐 2006년부터 연속해서 Developer Technologies 분야 Microsoft MVP로 활동 중이다. 수많은 .NET 프로젝트를 거쳐 현재는 하와소(hawaso) 대표로 소프트웨어 개발과 강의를 병행하고 있다. TechDays, DevOpsCamp 등 다양한 기술 세미나의 스피커로 활동하며 다수의 책을 집필하고 번역에 참여했다. 또한, Taeyo.NET, ASP.NET Korea 커뮤니티 운영진이기도 하다.블로그  닷넷코리아 dotnetkorea.com유튜브  비주얼아카데미 youtube.com/c/VisualAcademy
1부 C# 준비
01 C# 

Unity 개발을 하기 전에 C#은 어떤 녀석인가알기 위해 구매했던 책입니다.
이 책을 읽으면서 크게 불편했던 점은없었습니다.

다만 이런 내용을 처음 보는 사람이 이해할 수 있을까? 싶었던 부분이 조금 있었습니다.
본지 좀 되어서 어디에 있었는지는 기억이 나지 않습니다만...
그래도 책 전체에서 보면아주 작은 부분에 불과합니다.

인터넷 검색과 함께라면 어느...더보기
Unity 개발을 하기 전에 C#은 어떤 녀석인가 알기 위해 구매했던 책입니다.이 책을 읽으면서 크게 불편했던 점은 없었습니다.다만 이런 내용을 처음 보는 사람이 이해할 수 있을까? 싶었던 부분이 조금 있었습니다.본지 좀 되어서 어디에 있었는지는 기억이 나지 않습니다만...그래도 책 전체에서 보면 아주 작은 부분에 불과합니다.인터넷 검색과 함께라면 어느정도 해결이 가능하겠지요.
배송 엄청 빨라요 주문하고 그 다다음날 바로 왔어요.사은품으로 보낸 노란색 스터디플래너도 잘 왔고, 책도 구겨지거나 손상된 부분 없이 상태 멀쩡해요!책 내용은 목차를 봤을 때 기본문법부터 시작해서 활용, 확장까지 잘 나와있는 것 같아서 주문했어요.특히 8.0버전 책이 별로 없었는데.. 최신 버전 문법까지 잘 설명되어있네요! 처음 공부하는 분도 괜찮고 한...더보기
배송 엄청 빨라요 주문하고 그 다다음날 바로 왔어요.사은품으로 보낸 노란색 스터디플래너도 잘 왔고, 책도 구겨지거나 손상된 부분 없이 상태 멀쩡해요!책 내용은 목차를 봤을 때 기본문법부터 시작해서 활용, 확장까지 잘 나와있는 것 같아서 주문했어요.특히 8.0버전 책이 별로 없었는데.. 최신 버전 문법까지 잘 설명되어있네요! 처음 공부하는 분도 괜찮고 한번 전체적인 정리가 필요한 분도 괜찮을 것 같아요
비전공자 및 초보자도 이해하기 쉽게 유튜브에 무료 강의가 있어서 좋아요.그 외에도 책으로 어떻게 공부를 해야 하는지도 영상에서 강사님이 자세히 설명해주셔서 정말 좋아요.그리고 단순히 이론적인 내용이 아니라 예제로 수업을 진행하는 것이 정말 맘에들었어요.책에도 나오

# 47 신호와 시스템 : 저자소개, 출판사 리뷰, 추천사, 회원리뷰 없음 [600]

In [56]:
url = 'http://www.yes24.com/Product/Goods/8533856'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")
print("회원리뷰없음")



연속/이산 시간 신호와 시스템의 내용들을 쉽게 이해할 수 있도록 돕는다. 연속 시간 신호와 시간 시스템의 개괄적인 내용을 설명하고, 이를 확장하여 적용하는 과정까지 담고 있다.                    

없음
저자 서문 
역자 서문 

제1장 신호의 표현(Representing Signals) 
제2장 연속 시간 시스템(Continuous-Time Systems) 
제3장 푸리에 급수(Fourier Series) 
제4장 푸리에 변환(The Fourier Transform) 
제5장 라플라스 변환(The Laplace Transform) 
제6장 이산 시간 시스템(Discrete-Time Systems) 
제7장 이산 시간 시스템의 푸리에 해석(Fourier Analysis of Discrete-Time Systems) 
제8장 Z-변환(The Z-Transform) 
제9장 이산 푸리에 변환(The Discrete Fourier Transfrom) 
제10장 아날로그와 디지털 필터의 설계(Design of Analog and Digital Filters) 

부록 A 복소수(Complex Numbers) 
부록 B 수학적 관계(Mathematical Relations) 
부록 C 기본 행렬 이론(Elementary Matrix Theory) 
부록 D 부분 분수(Partial Fraction) 

참고문헌 
찾아보기
없음
없음
회원리뷰없음


# 48 선량한 차별주의자 [300]

In [57]:
url = 'http://www.yes24.com/Product/Goods/76470464'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/76470464?goodsSortNo=001009&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277125237'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



선량한 차별주의자들의 세상에서 
평등을 외치는 당신을 위한 안내서 

혐오와 차별은 잡초처럼 자란다. 조금만 신경 쓰지 않으면 온 사회에 무성해진다. 사람들은 때로 아주 작은 차별은 무시해도 되고, 심지어 다수에게 유리한 차별은 합리적인 차등이라고 이야기하며, 차별에 대한 문제제기나 시정조치를 역차별이라고 공격하기도 한다. 이런 말을 하는 사람들은 심각한 혐오주의자나 차별주의자가 아니다. 바로 나, 당신, 우리일 수 있다. 

 『선량한 차별주의자』는 평범한 우리 모두가 ‘선량한 차별주의자’일 수 있다고 말하는 도발적인 책이다. 저자인 김지혜 교수(강릉원주대 다문화학과)는 차별의 사각지대에 놓인 이들을 직접 찾아가는 현장활동가이자, 통계학·사회복지학·법학을 넘나드는 통합적인 시각을 바탕으로 국내의 열악한 혐오·차별 문제의 이론적 토대를 구축하는 데 전념해온 연구자다. 현장과 밀착한 인권·혐오문제 연구를 진행해온 연구자답게 이번 책에서 쉽고 재미있는 대중적 글쓰기를 선보인다. 인간 심리에 대한 국내외의 최신 연구, 현장에서 기록한 생생한 사례, 학생들과 꾸준히 진행해온 토론수업과 전문가들의 학술포럼에서의 다양한 논쟁을 버무려 우리 일상에 숨겨진 혐오와 차별의 순간들을 생생하게 담아냈다. 

은밀하고 사소하며 일상적이고 자연스럽게 벌어지는 일들 속에서 ‘선량한’ 우리가 놓치고 있던 ‘차별과 혐오의 순간’을 날카롭게 포착해내는 이 책의 메시지는 분명하다. 선량한 마음만으로 평등은 이루어지지 않으며, 익숙한 질서 너머의 세상을 상상하고 모두가 평등한 세상을 조직해가자고 제안한다. 차별을 당하면서도 작은 문제제기조차 해보지 못한 사람들부터 소위 프로불편러까지, 차별과 혐오의 시대에 지친 현대인들이 꼭 읽어야 할 책이다.

                                    

강릉원주대학교 다문화학과에서 소수자, 인권, 차별에 관해 가르치고 연구한다. 이주민, 성소수자, 아동·청소년, 홈리스 등 다양한 소수자 관련 현안에 관심을 가지고 현장과 밀접한 연

# 49 (사례와 함께하는) 회계원리 : 추천사 없음 [300]

In [58]:
url = 'http://www.yes24.com/Product/Goods/105787454'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/105787454?goodsSortNo=001004&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277162001'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



회계가 경영의 기초가 되거나 우리 삶과 밀접하게 연결된 중요한 지식을 가르치는 학문임에도, 회계를 공부하는 많은 학생이 지루함이나 어려움을 느끼는 것이 현실이다. 그 이유는 회계가 경영학의 다른 분야보다 더 많은 시간을 투입하여 공부해야 하고, 특히 논리적인 사고가 필요한 분야이기 때문일 것이다. 또한 한국의 회계교육이 원리에 대한 이해나 사례보다는, 발생한 거래를 회계장부에 정확하게 기록하는 것에 치우쳐 있다는 것에 이유가 있을 것이다. 이러한 배경에 『사례와 함께하는 회계원리』는 많은 학생이 회계입문과정에서 회계공부에 대한 흥미를 잃어버리고 포기하는 회계교육의 현실의 아쉬움을 줄이기 위한 목적에서 쓰였다.다른 교재들과 구분되는 『사례와 함께하는 회계원리』의 가장 중요한 특징은 각 장마다 상당한 시간과 노력을 들여 오랫동안 수집한 회계에 대한 재미있는 역사적 배경이나 관련 이야기들을 소개하고, 해당 장에서 설명한 지식의 배경이나 그와 관련되어 최근 벌어졌던 사례들도 제시한 것에 있다. 이를 통해 학생은 역사 속에서 회계의 중요성을 배울 수 있고, 교과서로 학습한 회계지식이 실무에서 어떻게 사용되고, 어떤 사건들이 관련하여 발생했는지 알 수 있을 것이다. 현실과 이론이 서로 어떻게 연결되는지를 깨우쳐 공부할 수 있도록 한 것이다.                    

서울대학교 경영대학 학부와 석사과정을 모두 수석으로 졸업했으며 서울대학교 총장상을 수상했다. 미국 일리노이 주립대학과 버클리대에서 경제학석사, 경영학 박사학위를 취득했다. 홍콩과기대학에서 교수로 지내는 동안 6년 연속 최고강의상을 수상했으며, 2006년 서울대학교 경영대학 교수로 부임했다. 2007년 서울대학교 우수연구상과 우수강의상을 최초로 동시에 수상했으며, 그 이후에도 다수 수상했다. 현재 서울대학교 MBA, 최고경영자과정AMP, CFO 전략과정 등에서 강의를 하고 있으며, 한국은행 근무, 플로리다주립대 조교수를 역임하였다. 이 외에도 Journal of Financial Economic

# 50 디지털 디자인 : 출판사 리뷰, 추천사 없음 [600]

In [59]:
url = 'http://www.yes24.com/Product/Goods/89386194'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/89386194?goodsSortNo=001004&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277203343'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))

없음
『디지털 디자인』의 저자이다.
1장 디지털 시스템과 2진수
2장 부울 대수와 논리 게이트
3장 게이트 레벨 최소화
4장 조합 논리
5장 동기식 순차 논리
6장 레지스터와 카운터
7장 메모리 및 프로그래머블 논리
8장 레지스터 전송 레벨 설계
9장 표준 IC 및 FPGA를 사용한 실험
10장 표준 그래픽 기호
없음
없음
[도서] 디지털 디자인은 퍼스트북 출판사에서 출간 된 책으로M.Morris Mano,Michael D.Ciletti 가 저서하고 예윤혜,김경백이 번역한 책입니다. 학교 수업때문에 구매하게 되었습니다. 책 번역이 잘되어있어서 책의 내용을 이해하기 쉬웠습니다.^^ 왜 대부분의 학교측에서 이 책을 전공 교재로 사용하는지 알 것 같습니다. 초급자도 이해하기 쉬워요더보기
[도서] 디지털 디자인은 퍼스트북 출판사에서 출간 된 책으로 M.Morris Mano,Michael D.Ciletti 가 저서하고 예윤혜,김경백이 번역한 책입니다. 학교 수업때문에 구매하게 되었습니다. 책 번역이 잘되어있어서 책의 내용을 이해하기 쉬웠습니다.^^ 왜 대부분의 학교측에서 이 책을 전공 교재로 사용하는지 알 것 같습니다. 초급자도 이해하기 쉬워요


# 51 파이썬 알고리즘 인터뷰 [600]

In [60]:
url = 'http://www.yes24.com/Product/Goods/91084402'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/91084402?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277350515'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



코딩 테스트와 인터뷰를 준비하는 취준생과 이직자를 위한 알고리즘 문제 풀이 완벽 마스터!
세계 최고 온라인 문제 풀이 사이트인 리트코드(LeetCode)의 기출문제 풀이와 분석!

『파이썬 알고리즘 인터뷰』는 200여 개가 넘는 일러스트를 통해 알고리즘과 자료구조 이론을 한눈에 쉽게 익힐 수 있음은 물론, 파이썬으로 구현하는 코드 풀이를 통해 성능 최적화 기법, 파이썬의 핵심 문법과 다양한 실용 테크닉까지 배울 수 있는, 초보 개발자와 숙련 개발자 모두를 위한 책이다. 마이크로소프트와 구글을 필두로 전 세계 개발자 채용의 첫 관문이 된 코딩 인터뷰! 주요 기업에서 실시하는 코딩 테스트의 기출 문제를 분석하고 상세한 문제 풀이와 해설을 통해 취업과 이직에 한발짝 다가서자. 이 책은 현업과 실무에 유용한 주요 알고리즘 이론을 깊숙이 이해하고, 파이썬의 핵심 기능과 문법까지 상세하게 이해할 수 있는 취업용 코딩 테스트를 위한 완벽 가이드다. 이 책을 다 읽고 나면, “왜 알고리즘을 공부하는가?”라는 질문에 대한 해답을 얻을 것이다.

                                    

인공지능 엔지니어. 현대자동차의 인공지능 연구조직인 AIRS에서 기술 리더를 맡고 있다. 이전에는 카카오에서 챗봇을, 다음커뮤니케이션에서 검색엔진을 만들며 검색에서 빅데이터, 인공지능으로 이어지는 디지털 기술을 두루 경험했다. 카카오 코딩 테스트 출제 위원이었고 현대자동차 연구개발 채용의 기술 면접관으로 활동하며 오랫동안 IT 직군의 인재를 발굴하는 일도 진행했다. 2020년에 출간한 『파이썬 알고리즘 인터뷰』는 네이버, 카카오 등 국내 최고의 IT 기업에 취업하기 위해 반드시 읽어야 하는 필수 지침서로 통한다.
[1부  코딩 인터뷰]

1장  코딩 인터뷰
코딩 인터뷰를 위한 온라인 테스트 플랫폼
국내 기업의 코딩 테스트 플랫폼 활용 현황
온라인 코딩 테스트의 사전 준비사항
화이트보드 코딩 인터뷰

2장  프로그래밍 언어 선택
경진대회 통계로 알아본 언어 선호도
프

# 52  클루지 [100]

In [61]:
url = 'http://www.yes24.com/Product/Goods/3142428'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")
url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/3142428?goodsSortNo=001014&resourceKeyGb=01&goodsStateGb=05&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277411075'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



10억 연봉 유튜버 ‘자수성가 청년’의 인생을 바꾼 책“더 나은 의사결정을 원한다면 반드시 이 책을 읽어라!”생각의 역사를 뒤집는 기막힌 발견『Kluge 클루지』. 23살에 MIT에서 뇌와 인지과학 박사 학위를 취득하고 30살의 나이에 종신 교수(tenure)가 된 스타 학자 개리 마커스 교수의 화제작이다. 저자가 주장하는 핵심 개념은 ‘진화의 관성(evolutionary inertia)'이다. 인간의 진화는 완벽한 체계를 만들기 위해 처음부터 다시 시작하는 것이 아니라, 이미 존재하는 것에 계속 ‘땜질’을 해가는 속성을 지녔다는 것이다. 그러므로 인간은 진화론적으로 충분히 완성되지 않은 존재이며, 우리의 뇌는 수만 년 동안 지금 당장의 생존을 최대한 추구하도록 설계된 비합리적인 컴퓨터라는 것이다. 이 책은 부조리한 일상과 혼란스러운 세상을 파헤치고 인간 본성의 불가사의한 측면을 조명할 뿐만 아니라 우리의 불완전함 속에서 생각의 무기를 찾아낼 수 있도록 명쾌한 해답을 제시한다. 저자 개리 마커스는 생각의 함정에서 생각의 무기를 찾아내는 지혜를 선보인다. 우리 내면의 클루지를 활용해서 어떻게 우리들의 세계를 개선시킬 수 있는지 놀라울 정도로 효과적인 방법들을 제시한다. 경험적 과학적으로 증명된 13가지 제안이 그것이다. 개리 마커스는 말한다. “진화는 우리에게 생각할 수 있는 능력을 부여했다. 그러나 그 생각이 오류가 없다고 보장하지는 않는다.” 『클루지』는 ‘생각하는 사람’으로 잘 살기 위한 소중한 단서와, 불완전하지만 고귀한 마음을 최대한 활용하는 독특한 기회를 제공한다.                    

뇌과학과 진화심리학, 언어학을 넘나들며 인간 마음의 기원을 연구하는 세계적인 인지과학자. 뉴욕대학교 심리학·신경과학 명예교수. 햄프셔대학교에서 인지과학으로 석사 학위를, MIT에서 스티븐 핑커 교수의 지도 아래 뇌과학을 연구하여 박사 학위를 마쳤다. 1996년, 전도유망한 젊은 심리학자에게 수여되는 로버트 판츠상을 수상했으며 머신러닝 스타트업인 지오

# 53 운영체제 : 출판사 서평, 추천사 없음 [600]

In [62]:
url = 'http://www.yes24.com/Product/Goods/89496122'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/89496122?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277472228'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))

없음
『운영체제』 저자이다.
Part 1 개관

Part 2 프로세스 관리

Part 3 프로세스 동기화

Part 4 메모리 관리

Part 5 저장장치 관리

Part 6 파일 시스템

Part 7 보안과 보호

Part 8 진보된 주제

Part 9 사례 검토

Part 10 부록

 찾아보기

없음
없음
[도서] 운영체제 리뷰 입니다. Abraham Silberschatz,Peter Baer Galvin,Greg Gagne 공저/박민규 역 저 입니다. 출판사는 퍼스트북 입니다. 2020년 2월에 출간되었네요

공룡책으로 유명한 운영체제를 구매해봤어요 실버바치 책이 가장 유명해서 구매했고 공부하기 좋아요 운영체제가 어려운 내용이지만 이해가 잘 되었고 번역서가 더 어렵긴 하네요더보기
[도서] 운영체제 리뷰 입니다. Abraham Silberschatz,Peter Baer Galvin,Greg Gagne 공저/박민규 역 저 입니다. 출판사는 퍼스트북 입니다. 2020년 2월에 출간되었네요공룡책으로 유명한 운영체제를 구매해봤어요 실버바치 책이 가장 유명해서 구매했고 공부하기 좋아요 운영체제가 어려운 내용이지만 이해가 잘 되었고 번역서가 더 어렵긴 하네요
교수님 추천으로 운영체제 공룡책 구매했습니다 이거 하나 사서 한번 정독하면 흐름은 파악할수 있을거라고 하시더군요 그만큼 자세하고 좋은 개념서입니다 운영체제를 처음 접하시는 분이라면 한권 사서 쭉 읽어보시는걸 추천합니다.. 그런데 전공서적이 항상 그렇듯이 글자가 빽빽하고 보기 좋게 편집된 책은 아닙니다 컬러풀한 책도 아니기에 최근 아이티 교육 도...더보기
교수님 추천으로 운영체제 공룡책 구매했습니다 이거 하나 사서 한번 정독하면 흐름은 파악할수 있을거라고 하시더군요 그만큼 자세하고 좋은 개념서입니다 운영체제를 처음 접하시는 분이라면 한권 사서 쭉 읽어보시는걸 추천합니다.. 그런데 전공서적이 항상 그렇듯이 글자가 빽빽하고 보기 좋게 편집된 책은 아닙니다 컬러풀한 책도 아니기에 최근 아이티 교육 도서만 접한 분이라

# 54 전자회로 : 출판사 리뷰, 추천사 없음 [600]

In [63]:
url = 'http://www.yes24.com/Product/Goods/69985174'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/69985174?goodsSortNo=001004&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277614858'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))

없음
『전자회로』, 『최신 디지털 공학』의 저자이다.
머리말 
역자 서문 

1 반도체 개요 
2 다이오드와 그 응용 
3 특수목적 다이오드 
4 바이폴라 접합 트랜지스터 
5 트랜지스터 바이어스 회로 
6 BJT 증폭기 
7 BJT 전력증폭기 
8 전계 효과 트랜지스터(FETs) 
9 FET 증폭기와 스위칭 회로 
10 증폭기의 주파수 응답 
11 사이리스터 
12 연산증폭기 
13 기본 연산증폭기 회로 
14 특수 목적 집적회로 
15 능동 필터 
16 발진기 
17 전압조정기 

연습문제 홀수번호 해답 
용어 정리 
찾아보기
없음
없음
전자회로 전공서 나름 최신책이라 잘 나와있고 풀컬러에 어려운 전자회로를 잘 이해시켜주는 책. 그래도 유x브 같은 영상으로 이해하는게 더 와닿겠지만 책을 참고하며 함께 학습하면 더욱 도음되는건 사실이다. 이 책보다도 더 쉬운책을 먼저 학습하면 좋을거 같다. 그리고 전자먼저 배우기보다는 회로이론 부터 떼고 와야 이해가 훨씬 수월하다 전자회로책 리뷰....더보기
전자회로 전공서 나름 최신책이라 잘 나와있고 풀컬러에 어려운 전자회로를 잘 이해시켜주는 책. 그래도 유x브 같은 영상으로 이해하는게 더 와닿겠지만 책을 참고하며 함께 학습하면 더욱 도음되는건 사실이다. 이 책보다도 더 쉬운책을 먼저 학습하면 좋을거 같다. 그리고 전자먼저 배우기보다는 회로이론 부터 떼고 와야 이해가 훨씬 수월하다 전자회로책 리뷰. 그림
전자회로의 리뷰

학교 수업의 주교재로 선택되어 구입하게되었다. 이 책은 영문판을 가지고 있어서 비교가 가능했는데,전체적으로 번역이 매끄럽게 되어있다는 느낌이 들어보기 정말 편했다. 또한 완성도가 매우 높은 책이라 이해도 쏙쏙 되었다. 가장 만족스러운 부분은 책의 순서이다. 전자회로를처음 접해보는 사람의 입장으로 보자면 개념부터 차근차근 설명해...더보기
전자회로의 리뷰학교 수업의 주교재로 선택되어 구입하게되었다. 이 책은 영문판을 가지고 있어서 비교가 가능했는데, 전체적으로 번역이 매끄럽게 되어있다는 느낌이 들어 보기 정

# 55 파이썬 머신러닝 완벽 가이드 : 추천사 없음 [600]

In [64]:
url = 'http://www.yes24.com/Product/Goods/108824557'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/108824557?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277671007'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



자세한 이론 설명과 파이썬 실습을 통해 머신러닝을 완벽하게 배울 수 있다!『파이썬 머신러닝 완벽 가이드』는 이론 위주의 머신러닝 책에서 탈피해, 다양한 실전 예제를 직접 구현해 보면서 머신러닝을 체득할 수 있도록 만들었습니다. 캐글과 UCI 머신러닝 리포지토리에서 난이도가 있는 실습 데이터를 기반으로 실전 예제를 구성했고, XGBoost, LightGBM, 스태킹 기법 등 캐글의 많은 데이터 사이언스에서 애용하는 최신 알고리즘과 기법을 상세하게 설명했다.이번 개정2판에서는 최신 사이킷런 버전(1.0.2)을 포함해 책에서 사용되는 모든 라이브러리를 최신 버전으로 업그레이드한 실습 코드를 구현하고, 다양한 유형의 하이퍼파라미터를 가지는 XGBoost나 LightGBM 모델의 최적 하이퍼파라미터 튜닝을 위한 베이지안 최적화 기법 적용 실습을 제공한다. 또한 머신러닝 관련 데이터 분석에 널리 쓰이는 시각화 라이브러리인 matplotlib과 seaborn의 활용법을 다룬 장을 새롭게 추가했다.                    

엔코아 컨설팅, 한국 오라클을 거쳐 현재 kx systems에서 고성능 인메모리 DB인 kdb+의 Machine Learning 컨설턴트로 근무 중이다. 지난 20년간 50여 개 이상의 주요 고객사에서 데이터컨설팅 분야에 매진해 왔으며, 최근 몇 년간은 AI 기반의 Advanced Analytics 분야에 집중하고 있다.  직접 구현해 보지 않으면 절대 이해하지 못하는 평범한 두뇌의 소유자이며, 절망적인 프로젝트에 참여해 자기학대적인 노력으로 문제를 해결하는 이상한 성격의 소유자이기도 하다.
 1장: 파이썬 기반의 머신러닝과 생태계 이해

01. 머신러닝의 개념
___머신러닝의 분류
___데이터 전쟁
___파이썬과 R 기반의 머신러닝 비교
02. 파이썬 머신러닝 생태계를 구성하는 주요 패키지
___파이썬 머신러닝을 위한 S/W 설치
03. 넘파이
___넘파이 ndarray 개요
___ndarray의 데이터 타입
___ndarray를 편리하

# 56 팩트풀니스 [300]

In [65]:
url = 'http://www.yes24.com/Product/Goods/69724044'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/69724044?goodsSortNo=001014&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277723544'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



빌 게이츠가 미국 모든 대학 졸업생에게 직접 선물한 화제의 책
강력한 사실을 바탕으로 세상을 정확하게 바라보는 방법을 담은 혁명적 저작

전 세계적으로 확증편향이 기승을 부리는 탈진실의 시대에, 막연한 두려움과 편견을 이기는 팩트의 중요성을 일깨우는 세계적 역작! 세계를 이해하기 위한 13가지 문제에서 인간의 평균 정답률은 16%, 침팬지는 33%. 우리는 왜 침팬지를 이기지 못하는가? 똑똑하고 현명한 사람일수록 세상의 참모습을 정확히 알지 못한다! ‘느낌’을 ‘사실’로 인식하는 인간의 비합리적 본능 10가지를 밝히고, 우리의 착각과 달리 세상이 나날이 진보하고 있음을 명확한 데이터와 통계로 증명한 놀라운 통찰. 세상을 바라보는 방식을 바꾸고 미래의 위기와 기회에 대처하기 위해 반드시 읽어야 할 필독서이다.

책은 출간과 동시에 세계 지성계를 사로잡으며 뜨거운 화제를 모았다. 버락 오바마 전 대통령은 반드시 읽어야 할 책 목록 5권 중에 하나로 추천했다. 스티븐 핑커 하버드대학 심리학과 교수는 풍부한 데이터를 통해 우리의 인지과정이 어떻게 우리를 잘못된 길로 이끌 수 있는지를 합리적으로 설명한 책으로, [네이처]는 읽는 것만으로도 우리의 세계관이 완전히 뒤바뀔 거라며 극찬했다. 또한 [옵저버]는 금세기 최고의 책으로 선정했고, [뉴욕타임스], [월스트리트저널], [가디언], [선데이타임스] 등 유수 언론의 베스트셀러에 오르며 출간 이후 6개월 만에 100만부를 돌파하는 기록을 세웠다.

전 세계적으로 확증편향이 기승을 부리는 탈진실(post-truth)의 시대에, 『팩트풀니스』는 막연한 두려움과 편견을 이기는 팩트의 중요성을 일깨우는 역작이다. 빈곤, 교육, 환경, 에너지, 인구 등 다양한 영역에서 우리가 생각하는 세계와 실제 세계의 간극을 좁히고 선입견을 깨는 통찰을 제시한다. 우리의 편견과 달리 세상이 나날이 진보하고 있음을, 사실에 충실한 명확한 데이터와 통계로 이를 낱낱이 증명한다.

빌 게이츠가 사회로 진출하는 청춘에게 이 책을 선물한 이유는, ‘

팩트풀니스 첫번째 테스트에 4문제를 맞추고 기존에 내가 습득했던 정보들이 잘못된 정보가 많다는 생각을 하게 되었다. TV를 보면 유니세프 광고나 후원광고 등에서 기아에 힘든 아이들,

그런 모습들이 아직도 어려운 곳이 많다는 생각을 하게 만드는게 아닌가 하는 생각이 들었고

후원을 받는 입장에서는 절박한 상황 등을 계속 연출하여야 하는 상황,

개발도상...더보기
팩트풀니스 첫번째 테스트에 4문제를 맞추고 기존에 내가 습득했던 정보들이 잘못된 정보가 많다는 생각을 하게 되었다. TV를 보면 유니세프 광고나 후원광고 등에서 기아에 힘든 아이들,그런 모습들이 아직도 어려운 곳이 많다는 생각을 하게 만드는게 아닌가 하는 생각이 들었고후원을 받는 입장에서는 절박한 상황 등을 계속 연출하여야 하는 상황,개발도상국과 선진국 우리는 잘 살고 아시아나 아프리카 나라는 못 산다는 교육을 받고 자라난내가  남을 낮춰야 내가 돋보이는 그런 현실을 생각하게 만들었다.그런 교육등을 꾸준히 받고 또한 미디어에 노출되어 무의식적으로 그렇게 판단하는 것이 마치미국이나 유럽에서 우리나라를 식민지배와 전쟁으로 원조를 받았던 나라라고 생각하는 것과같다고 생각한다.본능을 제어하는 여러 방법들 중 비난 본능이 가장 인상 깊었고 실제 자주 사용하는 것 같다.어떤 일에 결과를 판단할 때 비난이나 칭찬을 가장 쉬운선택 시스템을 잘 봐야 한다는 생각을 가지고 있지만 실제적으로는 비난과 칭찬을 사용하기 편해 자주 사용하는 것과 같다.비난 보다는 칭찬을 자주 사용해야 된다고 생각되어지지만 잘 되지는 않는다.미디어 리터러쉬에 대한 내용들도 인상 깊었다.
사람들은 양극단의 것을 치우쳐 봅니다.저자는 양극단 사이에 간극이 있다는 것을 인정해야 한다고 말합니다.사람들은 별것 아닌 상황에서 극단적인 공포를 느낄 수 있고 머릿속이 공포에 사로잡혀 있으면 사실을 제대로 바라보기 어렵다고 말합니다.또한 사건의 원인을 찾을 때는 마녀사냥하듯 파편하나를 파고들거나 부풀려서는 안되고 누군가에게 책임전가 해서도...더보기
사람들

# 57 서울 자가에 대기업 다니는 김 부장 이야기 [100]

In [66]:
url = 'http://www.yes24.com/Product/Goods/103286195'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/103286195?goodsSortNo=001022&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277761003'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



2021년 대한민국 직장생활과 부동산에 관한 현실을 적나라하게 표현한 하이퍼리얼리즘 스토리부동산 카페를 비롯, 각종 커뮤니티에서 크게 화제가 되어 30일 만에 조회수 1,000만을 기록해 주요언론 톱기사를 장식한다. 강제은퇴와 월급노예에 처한 직장인들의 실태, ‘있는 자와 없는 자’로 전국을 양분화한 대한민국 부동산에 얽힌 이야기를 김 부장, 송 과장, 정 대리, 권 사원 등의 생생한 캐릭터를 통해 적나라한 팩션 형태로 풀어내어 ‘2021판 미생’ ‘코인급 중독’이란 별명을 얻었다. 대단한 재미와 공감력을 인정받아 책 출간은 물론이고 웹툰, 드라마 제작까지 진행중이다.                      

대한민국의 평범한 직장인. 12년 차 과장이다. 매일 아침 4시 30분에 일어나 한 시간씩 글을 써서 온라인에 올린 것이 화제가 되어 《서울 자가에 대기업 다니는 김 부장 이야기》를 단행본으로 출판했고, 독자들의 요청과 응원에 힘입어 2권 ‘정 대리 · 권 사원 편’과 3권 ‘송 과장 편’을 출간했다. ‘김 부장 이야기’ 시리즈는 출판계를 넘어 방송계에까지 큰 화제를 몰고 와 현재 드라마와 웹툰으로 제작 중이다. ‘김 부장 이야기’ 드라마 대본 작업에 직접 참여함은 물론, 새로운 드라마도 집필하고 있다. ‘김 부장 이야기’ 시리즈 출간 이후 2년 만에 신작 《나의 돈 많은 고등학교 친구》를 들고 독자 곁을 찾아왔다.
추천의 글
저자의 글

김 부장의 평화로운 일상 
판도라의 상자
그놈이…… 건물주라고?
부동산 투자도 잘하는 대기업 부장
올 것이 왔구나 
“사장님, 이건 인생에 찾아온 3번의 기회 중 하나예요”
공황 장애 
풀썩 
대출 이자 지불할 날짜가 다가오고 있다
아내님 손바닥 안 
15도, 45도, 90도 
안녕히 가십쇼!

★★★★★ “너무너무 재미있다!단숨에 이 책을 읽었다. 이 작가의 필력은 최고다!”-브라운스톤(우석)-30일 만에 커뮤니티 조회수 1,000만조선일보 1면 톱장식드라마, 웹툰 제작 확정브라운스톤(우석), 신사임당, 배우 류수

# 58 작별인사 : 추천사 없음 [800]

In [67]:
url = 'http://www.yes24.com/Product/Goods/108887930'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/108887930?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277794214'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



누구도 도와줄 수 없는 상황, 혼자 헤쳐나가야 한다지켜야 할 약속, 붙잡고 싶은 온기 김영하가 『살인자의 기억법』 이후 9 년 만에 내놓는 장편소설 『작별인사』는 그리 멀지 않은 미래를 배경으로, 별안간 삶이 송두리째 뒤흔들린 한 소년의 여정을 좇는다. 유명한 IT 기업의 연구원인 아버지와 쾌적하고 평화롭게 살아가던 철이는 어느날 갑자기 수용소로 끌려가 난생처음 날것의 감정으로 가득한 혼돈의 세계에 맞닥뜨리게 되면서 정신적, 신체적 위기에 직면한다. 동시에 자신처럼 사회에서 배제된 자들을 만나 처음으로 생생한 소속감을 느끼고 따뜻한 우정도 싹틔운다. 철이는 그들과  함께  수용소를 탈출하여 집으로 돌아가기 위해 길을 떠나지만  그 여정에는 피할 수 없는 질문이 기다리고 있다.                     

1968년 강원도 화천에서 태어나 군인인 아버지를 따라 여러 지역을 옮겨 다니며 성장했다. 잠실의 신천중학교와 잠실고등학교를 졸업하고 연세대학교 경영학 학사와 석사를 취득했다. 한 번도 자신이 작가가 될 것이라고 생각하지 않았지만, 대학원에 재학 중이던 1990년대 초에 PC통신 하이텔에 올린 짤막한 콩트들이 뜨거운 반응을 얻는 것을 보고 자신의 작가적 재능을 처음으로 깨달았다. 서울에서 아내와 함께 살며 여행, 요리, 그림 그리기와 정원 일을 좋아한다.1995년 계간 [리뷰]에 「거울에 대한 명상」을 발표하며 작품 활동을 시작했다. 장편소설 『살인자의 기억법』, 『너의 목소리가 들려』, 『퀴즈쇼』, 『빛의 제국』, 『검은 꽃』, 『아랑은 왜』, 『나는 나를 파괴할 권리가 있다』, 소설집 『오직 두 사람』, 『무슨 일이 일어났는지는 아무도』, 『오빠가 돌아왔다』, 『엘리베이터에 낀 그 남자는 어떻게 되었나』, 『호출』, 여행에 관한 산문 『여행의 이유』와 『오래 준비해온 대답』을 냈고, 산문집 삼부작 『보다』, 『말하다』, 『읽다』 삼부작과 『랄랄라 하우스』 등이 있다. F. 스콧 피츠제럴드의 『위대한 개츠비』를 번역했다. 문학동네작가상 동인문학

# 59 (2021) 산업안전기사 : 추천사, 회원리뷰 없음 [600]

In [68]:
url = 'http://www.yes24.com/Product/Goods/96201044'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

print("회원리뷰 없음")



최근 출제 기준에 맞춘,산업안전기사 2021년 최신판!!이 책은 저자가 강단에서의 오랜 강의 경험을 토대로 틈틈이 준비하였던 자료를 바탕으로 새로운 출제 기준에 맞추어 출간하였기 때문에, 산업안전기사 필기시험을 앞둔 수험생과 산업 현장에서 실무에 종사하는 산업역군에게 도움을 줄 수 있을 것이다. 또한, 산업인력공단 출제 기준에 맞추어 매 단원마다 이론과 예상 문제를 수록하였고, 상세한 해설로 수험생들이 철저하게 시험을 준비할 수 있도록 하였다. 부록으로는 최근에 출제된 문제(2012년 3월∼2020년 9월)를 수록하여 수험생 스스로 출제 경향을 파악하며 시험에 앞서 문제에 대한 적응력과 응용력을 키울 수 있도록 하였다.                    

- 울산대학교 외래교수- 삼육대학교 외래교수- 호서대학교 외래교수- 한국폴리텍Ⅰ대학 겸임교수- 경남정보대학 외래교수
- 머리말
- 산업안전기사 출제 기준
- 시험안내
- NCS 안내

PART 01 안전관리론
  Chapter 01. 안전보건관리 개요
  Chapter 02. 재해 및 안전점검
  Chapter 03. 무재해 운동 및 보호구
  Chapter 04. 산업안전심리
  Chapter 05. 안전·보건교육
  Chapter 06. 산업안전관계법규
  
PART 02 인간공학 및 시스템 안전공학
  Chapter 01. 안전과 인간공학
  Chapter 02. 제 어
  Chapter 03. 작업과 인간공학
  Chapter 04. 작업환경관리
  Chapter 05. 시스템 위험분석
  Chapter 06. 결함수 분석법(FTA)
  Chapter 07. 안전성 평가(Safety Assessment)
  Chapter 08. 각종 설비의 유지관리

PART 03 기계위험방지기술
  Chapter 01. 기계의 안전 개념
  Chapter 02. 공작기계의 안전
  Chapter 03. 프레스 및 전단기의 안전
  Chapter 04. 기타 산업용 기계?기구
  Chapter 05. 운반기계 및

# 60 (2022 시나공) 정보처리기사 : 추천사 없음 [600]

In [69]:
url = 'http://www.yes24.com/Product/Goods/107477760'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 책소개
introduce_tags = soup.select('div.infoWrap_txtInner')
if introduce_tags:
    introduce = introduce_tags[0].text
    print(introduce)
else:
    print('없음')
print("===============================================")

# 저자소개
intro_tags = soup.select('span.author_info.info_origin')
if intro_tags:
    intro_text = intro_tags[0].text.strip()
    print(intro_text)
else:
    print('없음')
print("===============================================")

# 목차
info_tags = soup.select('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')
if info_tags:
    info_text = info_tags[0].text.strip()
    info_text = re.sub('<br/>', '\n', info_text)
    info_text = re.sub('<.*?>', '', info_text)
    print(info_text)
else:
    print('없음')
print("===============================================")

# 출판사리뷰
rewiew_pub_tags = soup.select('#infoset_pubReivew > div.infoSetCont_wrap > div.infoWrap_txt')
if rewiew_pub_tags:
    rewiew_pub_text = rewiew_pub_tags[0].text.strip()
    print(rewiew_pub_text)
else:
    print('없음')
print("===============================================")

# 추천평
res_tags = soup.select('#infoset_nomiCmt > div.infoSetCont_wrap > div.infoWrap_txt')
if res_tags:
    res_text = res_tags[0].text.strip()
    print(res_text)
else:
    print('없음')
print("===============================================")

url = 'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/107477760?goodsSortNo=001002&resourceKeyGb=01&goodsStateGb=05&goodsSetYn=N&goodsGb=01&Sort=1&PageNumber=1&Type=ALL&DojungAfterBuy=0&_=1685277877341'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
for review in soup.select('div.review_cont'):
    print(review.get_text(strip=True))



시나공 정보처리기사 실기는 NCS 학습 모듈을 가이드 삼아 자세한 설명과 충분한 예제를 더한 후 교재에 수록된 문제나 이론은 하나도 빼놓지 않고 이 분야에 전혀 기초가 없는 수험생의 눈높이에 맞춰 최대한 쉽게 설명했다.  ‘전 범위 무료 동영상 강의’, ‘2021년 최신기출문제와 동영상 풀이’, ‘실전 모의고사’, 오답노트를 작성할 수 있는 ‘기억상자’까지! 비전공자도 쉽게 학습하여 합격할 수 있도록 시나공이 함께 한다.                    

IT 서적을 기획하고 집필하는 출판 기획 전문 집단으로, 2003년부터 길벗출판사의 IT 수험서인 『시험에 나오는 것만 공부한다!』 시리즈를 기획부터 집필 및 편집까지 총괄하고 있다. 20여 년간 자격증 취득에 관한 교육, 연구, 집필에 몰두해 온 강윤석 실장을 중심으로 IT 자격증 시험의 분야별 전문가들이 모여 국내 IT 수험서의 수준을 한 단계 높이기 위한 다양한 연구와 집필 활동에 전념하고 있다.
『1권』

0장. 준비운동 

1장. 요구사항 확인

Section 001 소프트웨어 생명 주기 
Section 002 스크럼(Scrum) 기법 
Section 003 XP(eXtreme Programming) 기법 
Section 004 현행 시스템 파악 
Section 005 개발 기술 환경 파악 
Section 006 요구사항 정의 
Section 007 요구사항 개발 프로세스 
Section 008 요구사항 분석 
Section 009 요구사항 분석 CASE와 HIPO 
Section 010 UML(Unified Modeling Language)의 개요 
Section 011 UML-관계(Relationship) 
Section 012 UML-다이어그램 
Section 013 유스케이스(Use Case) 다이어그램 
Section 014 활동(Activity) 다이어그램 
Section 015 클래스(Class) 다이어그램 
Section 016 시퀀스(Sequence) 다이어그램 
Section